# 3D 시각화
전체 흐름   
Extractor: 질의에서 키워드 추출    
RAG Search: 키워드를 이용해 문서 검색   
Answer Generator: 검색 결과로부터 답변 생성    
Result Formatter: 결과 화면에 표시할 형식으로 가공   


----

### 전체 흐름 요약(Pipeline)
[extractor]    
    → 키워드 추출   
        → [rag_search]   
            → 문서 검색 + DB 조회 + 관련성 계산   
                → [answer_generator]   
                    → LLM 답변 생성   
                        → [result_formatter]   
                            → 최종 출력 포맷팅   
                                → END (완료)   

In [11]:
# ================================================================
# 노트북용 RAG + 3D 시각화 통합 코드
# ================================================================
# 이 코드는 RAG(Retrieval-Augmented Generation) 시스템을 Jupyter Notebook 환경에서 
# 실행하고, 처리 과정을 3D 시각화하는 통합 솔루션입니다.
# - 사용자가 입력한 질의에 대해 관련 문서를 검색하고, AI가 답변을 생성하는 과정을
#   실시간으로 3D 그래프와 차트로 모니터링할 수 있습니다.
# - 시스템은 4개의 에이전트로 구성되어 있으며, LangGraph로 연결되어 순차적으로 실행됩니다.
# ================================================================

from typing import TypedDict, List, Dict, Any
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate  # 프롬프트 템플릿 생성
from langchain_core.output_parsers import StrOutputParser  # LLM 출력 파싱
from langchain_community.llms import Ollama  # Ollama LLM
from langchain_community.chat_models import ChatOllama  # Ollama 챗 모델
from langchain_community.vectorstores import Chroma  # 벡터 데이터베이스
from langchain_community.embeddings import OllamaEmbeddings  # Ollama 임베딩
from langgraph.graph import StateGraph, END  # LangGraph 컴포넌트
import pymysql  # MySQL 데이터베이스 연동

# 시각화 관련
import plotly.graph_objects as go  # 3D 그래프 생성
import plotly.express as px  # 간편한 시각화
import numpy as np  # 수치 연산
import ipywidgets as widgets  # 노트북 위젯
from IPython.display import display  # 노트북 출력
import datetime  # 시간 처리

# ================================================================
# 1. 상태 정의 (AgentState)
# ================================================================
# - 에이전트 간에 전달되는 상태를 정의하는 자료 구조
# - query: 사용자 입력
# - keywords: 추출된 검색 키워드
# - search_results: 검색된 문서 목록 (파일명, 위치, 요약, 관련성 등)
# - context: 검색된 문서들의 결합 내용
# - result: 최종 생성된 답변
# ================================================================
class AgentState(TypedDict):
    query: str
    keywords: str
    search_results: List[Dict[str, Any]]
    context: str
    result: str

# ================================================================
# 2. 데이터베이스 접속 정보
# ================================================================
# - MySQL 데이터베이스 연결을 위한 설정
# - host: localhost
# - user: admin
# - password: 1qazZAQ!
# - db: final
# - charset: utf8mb4 (한글 지원)
# ================================================================
DB_CONFIG = {
    'host': 'localhost',
    'user': 'admin',
    'password': '1qazZAQ!',
    'db': 'final',
    'charset': 'utf8mb4'
}

# ================================================================
# 3. ChromaDB 및 LLM 설정
# ================================================================
# - ChromaDB: 벡터 데이터베이스로 문서 임베딩 저장
# - OllamaEmbeddings: 문서 임베딩용 Ollama 모델 (exaone3.5:2.4b)
# - Ollama: 텍스트 생성용 Ollama 모델 (exaone3.5:2.4b)
# - ChatOllama: 챗봇용 Ollama 모델 (exaone3.5:2.4b, temperature=0.1)
# ================================================================
CHROMA_PATH = "./rag_chroma/documents/title_summary/"  # ChromaDB 저장 경로
EMBEDDINGS = OllamaEmbeddings(model="exaone3.5:2.4b")  # 임베딩 모델
LLM = Ollama(model="exaone3.5:2.4b")  # LLM
CHAT_LLM = ChatOllama(model="exaone3.5:2.4b", temperature=0.1)  # 챗 모델

# ================================================================
# 4. 노트북용 3D 시각화 클래스 (RAGNotebookVisualizer)
# ================================================================
# - RAG 처리 과정을 3D로 시각화하는 클래스
# - 에이전트 상태, 검색 결과, 진행률을 실시간으로 보여줌
# ================================================================
class RAGNotebookVisualizer:
    def __init__(self):
        # 에이전트의 3D 공간 위치 (x, y, z)
        self.agent_positions = {
            'extractor': (0.0, 0.0, 0.0),       # 키워드 추출
            'rag_search': (2.0, 0.0, 1.0),      # 문서 검색
            'answer_generator': (4.0, 0.0, 2.0), # 답변 생성
            'result_formatter': (6.0, 0.0, 1.0)  # 결과 형식화
        }
        self.agent_names = list(self.agent_positions.keys())
        # 에이전트 초기 상태 (대기중)
        self.agent_status = {name: 'waiting' for name in self.agent_names}

        # 3D FigureWidget 생성
        self.fig3d = go.FigureWidget()
        self._init_3d_scene()  # 3D 장면 초기화

        # 관련성 막대 그래프 (검색 결과)
        self.bar_fig = go.FigureWidget(
            data=[go.Bar(x=[], y=[], text=[], textposition='auto')],
            layout=go.Layout(
                title='검색 결과 관련성 (%)',
                height=260,
                margin=dict(t=40)
            )
        )

        # 진행률 바 (에이전트 진행 상황)
        self.progress = widgets.FloatProgress(
            value=0.0,
            min=0.0,
            max=100.0,
            description='진행률:',
            bar_style='info'
        )

        # 로그 텍스트 영역 (처리 기록)
        self.log = widgets.Textarea(
            value='',
            placeholder='로그가 여기에 표시됩니다.',
            layout=widgets.Layout(width='100%', height='600px')
        )

        # 전체 UI 레이아웃 (3D + 차트 + 로그)
        left_panel = widgets.VBox([self.fig3d], layout=widgets.Layout(width='60%'))
        right_panel = widgets.VBox([self.bar_fig, self.progress, self.log], layout=widgets.Layout(width='40%'))
        self.container = widgets.HBox([left_panel, right_panel])

    def _init_3d_scene(self):
        """3D 장면을 초기화하고 에이전트 노드/연결선을 추가"""
        # 에이전트 위치 (x, y, z)
        xs = [pos[0] for pos in self.agent_positions.values()]
        ys = [pos[1] for pos in self.agent_positions.values()]
        zs = [pos[2] for pos in self.agent_positions.values()]

        # 에이전트 노드 (마커 + 텍스트)
        node_trace = go.Scatter3d(
            x=xs,
            y=ys,
            z=zs,
            mode='markers+text',  # 마커와 텍스트 함께 표시
            marker=dict(
                size=14,            # 마커 크기
                color=['gray'] * len(self.agent_names),  # 초기 색상 (회색)
                opacity=0.9,         # 투명도
                line=dict(width=2, color='DarkSlateGrey')  # 마커 테두리
            ),
            text=self.agent_names,   # 노드 텍스트
            textposition="top center",  # 텍스트 위치
            textfont=dict(size=12, color='black'),
            name="에이전트"
        )
        self.fig3d.add_trace(node_trace)

        # 에이전트 간 연결선 (에지)
        for i in range(len(self.agent_names) - 1):
            a = self.agent_positions[self.agent_names[i]]
            b = self.agent_positions[self.agent_names[i+1]]
            edge = go.Scatter3d(
                x=[a[0], b[0]],
                y=[a[1], b[1]],
                z=[a[2], b[2]],
                mode='lines',
                line=dict(color='lightgray', width=4),
                showlegend=False
            )
            self.fig3d.add_trace(edge)

        # 검색 결과 placeholder (업데이트 예정)
        search_trace = go.Scatter3d(
            x=[], y=[], z=[],
            mode='markers+text',
            marker=dict(
                size=[], 
                color=[], 
                colorscale='Viridis',  # 색상 척도
                cmin=0, 
                cmax=100, 
                opacity=0.8, 
                showscale=True,       # 색상 바 표시
                colorbar=dict(title='관련성 (%)')
            ),
            text=[], 
            textposition='top center',
            name='검색 결과'
        )
        self.fig3d.add_trace(search_trace)

        # 3D 레이아웃 설정
        self.fig3d.update_layout(
            title='🤖 RAG 처리 과정 3D 시각화 (Notebook)',
            scene=dict(
                xaxis=dict(title='처리 단계'),
                yaxis=dict(title='데이터 흐름'),
                zaxis=dict(title='복잡도'),
                camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))  # 3D 뷰 각도
            ),
            height=600,
            margin=dict(l=0, r=0, t=40, b=0),  # 마진
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)  # 범례
        )

    def show(self):
        """시각화 UI를 노트북에 표시"""
        display(self.container)

    def _append_log(self, text: str):
        """로그 메시지를 추가하고 표시"""
        # 현재 시간 포맷: HH:MM:SS
        timestamp = datetime.datetime.now().strftime("%H:%M:%S")
        self.log.value += f"[{timestamp}] {text}\n"

    def update_agent_status(self, agent_name: str, status: str, data: List[Dict[str, Any]] = None):
        """
        에이전트 상태를 업데이트하고 시각화를 갱신
        
        Args:
            agent_name: 업데이트할 에이전트 이름
            status: 'processing', 'completed', 'error', 'waiting'
            data: 검색 결과 데이터 (rag_search 에이전트 전용)
        """
        # 1. 로그에 상태 추가
        status_text = status.capitalize()
        self._append_log(f"{agent_name} -> {status_text}")

        # 2. 에이전트 상태 저장
        if agent_name in self.agent_status:
            self.agent_status[agent_name] = status

        # 3. 색상 매핑: 상태에 따른 색상
        def get_status_color(s):
            if s == 'processing': return 'yellow'
            elif s == 'completed': return 'green'
            elif s == 'error': return 'red'
            else: return 'gray'  # waiting

        # 4. 3D 노드 색상 업데이트
        colors = [get_status_color(self.agent_status[name]) for name in self.agent_names]
        self.fig3d.data[0].marker.color = colors

        # 5. 진행률 업데이트 (완료된 에이전트 비율)
        completed_count = sum(1 for s in self.agent_status.values() if s == 'completed')
        progress_percent = (completed_count / len(self.agent_status)) * 100
        self.progress.value = progress_percent

        # 6. 검색 결과가 있는 경우 시각화 업데이트
        if data and agent_name == 'rag_search':
            self.update_search_results(data)

def update_search_results(self, results: List[Dict[str, Any]]):
    """검색 결과를 3D 그래프와 막대 차트에 업데이트
    
    Args:
        results: 검색 결과 목록 (relevance, file_name, ...)
    """
    # 1. 관련성 순으로 내림차순 정렬
    results_sorted = sorted(results, key=lambda x: x.get('relevance', 0), reverse=True)
    
    # 2. 3D 검색 결과 업데이트
    base_x, base_y, base_z = self.agent_positions['rag_search']  # rag_search 위치
    n = len(results_sorted)
    xs, ys, zs, sizes, colors, texts = [], [], [], [], [], []
    radius = 1.5  # 원형 배치 반지름

    # 3. 검색 결과를 원형으로 배치
    for i, res in enumerate(results_sorted):
        angle = (i * 2 * np.pi) / n  # 각도
        # 3D 좌표: rag_search 위치를 중심으로 원형 배치
        x = base_x + radius * np.cos(angle)
        y = base_y + radius * np.sin(angle)
        z = base_z + (res.get('relevance', 0) / 100) * 2  # 관련성에 따른 높이
        
        xs.append(x)
        ys.append(y)
        zs.append(z)
        sizes.append(max(6, res.get('relevance', 0) / 4))  # 관련성에 따른 크기
        colors.append(res.get('relevance', 0))  # 색상 (관련성)
        # 텍스트: 파일명 + 관련성
        text = f"{res.get('file_name', '파일')}<br>{res.get('relevance', 0)}%"
        texts.append(text)

    # 4. 3D 검색 결과 trace 업데이트
    for trace in self.fig3d.data:
        if trace.name == '검색 결과':
            trace.x = xs
            trace.y = ys
            trace.z = zs
            trace.marker.size = sizes
            trace.marker.color = colors
            trace.text = texts
            break

    # 5. 막대 차트 업데이트 (관련성)
    file_names = [res.get('file_name', '') for res in results_sorted]
    relevances = [res.get('relevance', 0) for res in results_sorted]
    self.bar_fig.data[0].x = file_names
    self.bar_fig.data[0].y = relevances
    self.bar_fig.data[0].text = [f"{r}%" for r in relevances]
    self.bar_fig.data[0].marker.color = px.colors.sequential.Viridis  # 색상 척도

    # 6. 막대 차트 레이아웃 업데이트 (너비 조절)
    self.bar_fig.update_layout(
        xaxis_tickangle=-45,  # 파일명 45도 회전
        margin=dict(b=100)    # 파일명 간격 확보
    )

# ================================================================
# 5. 시각화 객체 생성
# ================================================================
# - 위에서 정의한 RAGNotebookVisualizer 인스턴스 생성
# - 노트북에서 show() 메소드를 호출해 UI 표시
# ================================================================
visualizer = RAGNotebookVisualizer()

# ================================================================
# 6. RAG 에이전트 함수
# ================================================================
# - 4개의 에이전트: 키워드 추출, RAG 검색, 답변 생성, 결과 형식화
# - 각 에이전트는 visualizer.update_agent_status()로 상태를 업데이트
# ================================================================

# 6.1. 키워드 추출 에이전트
def extractor_agent(state: AgentState):
    # 상태: processing
    visualizer.update_agent_status('extractor', 'processing')
    
    # 프롬프트 템플릿: 사용자 질의에서 키워드 추출
    keyword_prompt = PromptTemplate.from_template(
        """사용자의 질문에서 띄어쓰기 확인하고 찾고자 하는 키워드를 쉼표로 구분하여 출력하세요.
        벡터스토어 검색을 위한 최적의 키워드를 추출해주세요.
        \n질문: {query}"""
    )
    # 프롬프트 포맷팅
    formatted_prompt = keyword_prompt.format(query=state["query"])
    # LLM 호출
    keywords = LLM.invoke(formatted_prompt)
    
    # 상태: completed
    visualizer.update_agent_status('extractor', 'completed')
    # state 업데이트 (keywords 추가)
    return {**state, "keywords": keywords.strip()}

# 6.2. RAG 검색 에이전트
def rag_search_agent(state: AgentState):
    # 상태: processing
    visualizer.update_agent_status('rag_search', 'processing')
    
    try:
        # 1. ChromaDB 로드
        vectorstore = Chroma(persist_directory=CHROMA_PATH, embedding_function=EMBEDDINGS)
        # 2. 벡터스토어 검색 (k=10: 상위 10개)
        results = vectorstore.similarity_search_with_score(state["keywords"], k=10)

        if not results:
            # 검색 결과 없음
            visualizer.update_agent_status('rag_search', 'completed', [])
            return {**state, "search_results": [], "context": ""}

        # 3. 문서 중복 제거 (최고 유사도 유지)
        best_doc_info = {}
        for doc, score in results:
            doc_id = doc.metadata.get("id")  # 문서 ID
            if doc_id:
                content = doc.page_content.strip()
                if not content:  # 내용이 빈 경우 스킵
                    continue
                # 동일 문서 ID 중 가장 높은 유사도를 유지
                if doc_id not in best_doc_info or score < best_doc_info[doc_id][0]:
                    best_doc_info[doc_id] = (score, content)

        if not best_doc_info:
            visualizer.update_agent_status('rag_search', 'completed', [])
            return {**state, "search_results": [], "context": ""}

        # 4. 유사도 정규화 (0-100%로 변환)
        scores = [score for score, _ in best_doc_info.values()]
        min_score = min(scores)
        max_score = max(scores)

        # 5. MySQL에서 파일 메타데이터 조회
        conn = None
        search_results = []
        try:
            conn = pymysql.connect(**DB_CONFIG)
            cursor = conn.cursor(pymysql.cursors.DictCursor)

            for doc_id, (score, content) in best_doc_info.items():
                # MySQL: documents 테이블에서 문서 ID로 조회
                cursor.execute("SELECT * FROM documents WHERE id = %s", (doc_id,))
                row = cursor.fetchone()
                if not row:
                    continue  # 데이터 없을 경우 스킵

                # 6. 관련성 % 계산
                if min_score == max_score:
                    relevance = 100.0  # 모든 문서 동일 유사도
                else:
                    # min-max normalization: 0-100%로 변환
                    relevance = round((1 - (score - min_score) / (max_score - min_score)) * 100, 1)

                # 7. 검색 결과 목록에 추가
                search_results.append({
                    "relevance": relevance,
                    "file_name": row["file_name"],
                    "file_location": row["file_location"],
                    "summary": (row["summary"][:200] + "...") if row["summary"] else "요약 없음",
                    "title": row["title"],
                    "created_at": row["created_at"].strftime("%Y-%m-%d") if row["created_at"] else "N/A",
                    "doc_type": row["doc_type"],
                    "content": content
                })

            # 8. 관련성 순으로 정렬 (내림차순) 및 상위 10개
            search_results.sort(key=lambda x: x["relevance"], reverse=True)
            search_results = search_results[:10]
            # 9. 문서 내용 결합 (context)
            context = "\n\n".join([res['content'] for res in search_results])

        except Exception as e:
            # MySQL 오류 처리
            print(f"❌ MySQL 오류: {e}")
            visualizer.update_agent_status('rag_search', 'error')
            return {**state, "search_results": [], "context": ""}
        finally:
            if conn:
                conn.close()

        # 상태: completed + 검색 결과 데이터 전달
        visualizer.update_agent_status('rag_search', 'completed', search_results)
        return {**state, "search_results": search_results, "context": context}

    except Exception as e:
        # 일반 오류 처리
        print(f"❌ RAG 검색 오류: {e}")
        visualizer.update_agent_status('rag_search', 'error')
        return {**state, "search_results": [], "context": ""}

# 6.3. 답변 생성 에이전트
def answer_generator_agent(state: AgentState):
    # 상태: processing
    visualizer.update_agent_status('answer_generator', 'processing')
    
    # 검색 결과가 없는 경우
    if not state["search_results"]:
        visualizer.update_agent_status('answer_generator', 'completed')
        return {**state, "result": "관련 정보를 찾을 수 없습니다."}

    try:
        # 1. 프롬프트 템플릿
        prompt = ChatPromptTemplate.from_template(
            """다음 문서 내용들을 바탕으로 사용자 질문에 답변해 주세요.
            - 문서에 직접 언급된 내용만을 바탕으로 답변해야 합니다.
            - 정보가 없는 경우 "정보가 없습니다"라고 명확히 답변해 주세요.
            - 항상 공손하고 전문적인 어조를 유지해 주세요.

            ### 검색 결과 요약:
            {search_summary}

            ### 문서 내용:
            {context}

            ### 사용자 질문:
            {query}

            ### 답변:
            찾은 문서중에 몇번째 정보가 가장 관련이 높은지를 상세히 답변해 주세요.
            (몇번째, 무슨파일인지, 어디에 있는지, 요약은 무엇인지 등)
            """
        )

        # 2. 검색 결과 요약 생성
        search_summary = "\n".join([
            f"{i+1}순위 ({res['relevance']}%): {res['file_name']} ({res['doc_type']}) - {res['summary']}"
            for i, res in enumerate(state["search_results"])
        ])

        # 3. LLM 체인: 프롬프트 + LLM + 출력 파서
        chain = prompt | CHAT_LLM | StrOutputParser()
        result = chain.invoke({
            "search_summary": search_summary,
            "context": state["context"],
            "query": state["query"]
        })

        # 상태: completed
        visualizer.update_agent_status('answer_generator', 'completed')
        return {**state, "result": result.strip()}

    except Exception as e:
        print(f"❌ 답변 생성 오류: {e}")
        visualizer.update_agent_status('answer_generator', 'error')
        return {**state, "result": "답변을 생성하는 중에 오류가 발생했습니다."}

# 6.4. 결과 형식화 에이전트
def result_formatter_agent(state: AgentState):
    # 상태: processing
    visualizer.update_agent_status('result_formatter', 'processing')
    
    # 검색 결과가 없는 경우
    if not state["search_results"]:
        visualizer.update_agent_status('result_formatter', 'completed')
        return state

    try:
        # 1. 검색 결과 형식화 (Markdown 스타일)
        formatted_result = "\n🔍 검색 결과:\n"
        for i, res in enumerate(state["search_results"], 1):
            formatted_result += f"\n--- {i}순위 ({res['relevance']}%) ---\n"
            formatted_result += f"📄 파일명: {res['file_name']}\n"
            formatted_result += f"📁 위치: {res['file_location']}\n"
            formatted_result += f"📝 요약: {res['summary']}\n"
            formatted_result += f"🏷️ 유형: {res['doc_type']}\n"
            formatted_result += f"🕒 등록일: {res['created_at']}\n"

        formatted_result += f"\n💬 AI 답변:\n{state['result']}"

        # 상태: completed
        visualizer.update_agent_status('result_formatter', 'completed')
        return {**state, "result": formatted_result}

    except Exception as e:
        print(f"❌ 결과 형식화 오류: {e}")
        visualizer.update_agent_status('result_formatter', 'error')
        return state

# ================================================================
# 7. LangGraph 구성
# ================================================================
# - 에이전트들을 노드로 연결한 그래프 생성
# - 실행 순서: extractor → rag_search → answer_generator → result_formatter
# ================================================================
graph = StateGraph(AgentState)  # AgentState를 상태로 사용하는 그래프
graph.add_node("extractor", extractor_agent)  # 노드 추가
graph.add_node("rag_search", rag_search_agent)
graph.add_node("answer_generator", answer_generator_agent)
graph.add_node("result_formatter", result_formatter_agent)

# 시작 노드 설정
graph.set_entry_point("extractor")

# 노드 연결
graph.add_edge("extractor", "rag_search")
graph.add_edge("rag_search", "answer_generator")
graph.add_edge("answer_generator", "result_formatter")
graph.add_edge("result_formatter", END)  # END: 그래프 종료

# 그래프 컴파일
app = graph.compile()

# ================================================================
# 8. 실행 예시 (Jupyter Notebook)
# ================================================================
# 1) 시각화 UI 표시
visualizer.show()

# 2) RAG 파이프라인 실행
state = {
    "query": "기초생활수급자 차상위계층",  # 사용자 질의
    "keywords": "",            # 초기 키워드
    "search_results": [],      # 검색 결과
    "context": "",             # 문서 내용
    "result": ""               # 최종 답변
}

# 3) invoke()로 그래프 실행
result = app.invoke(state)

# 4) 최종 결과 출력
print("\n" + "="*50)
print("📊 RAG 처리 완료! 최종 결과:")
print("="*50)
print(result["result"])

# (visualizer는 에이전트 실행 도중 실시간으로 업데이트됩니다.)

    'data': [{'marker': {'color': [gray, gray, gray, gray],
     …

❌ RAG 검색 오류: 'RAGNotebookVisualizer' object has no attribute 'update_search_results'

📊 RAG 처리 완료! 최종 결과:
관련 정보를 찾을 수 없습니다.


# 추가 DEMO 

In [10]:
# ================================================================
# 노트북용 RAG + 3D 시각화 통합 코드
# ================================================================
# 이 코드는 RAG(Retrieval-Augmented Generation) 시스템을 Jupyter Notebook 환경에서 
# 실행하고, 처리 과정을 3D 시각화하는 통합 솔루션입니다.
# - 사용자가 입력한 질의에 대해 관련 문서를 검색하고, AI가 답변을 생성하는 과정을
#   실시간으로 3D 그래프와 차트로 모니터링할 수 있습니다.
# - 시스템은 4개의 에이전트로 구성되어 있으며, LangGraph로 연결되어 순차적으로 실행됩니다.
# ================================================================

from typing import TypedDict, List, Dict, Any
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate  # 프롬프트 템플릿 생성
from langchain_core.output_parsers import StrOutputParser  # LLM 출력 파싱
from langchain_community.llms import Ollama  # Ollama LLM
from langchain_community.chat_models import ChatOllama  # Ollama 챗 모델
from langchain_community.vectorstores import Chroma  # 벡터 데이터베이스
from langchain_community.embeddings import OllamaEmbeddings  # Ollama 임베딩
from langgraph.graph import StateGraph, END  # LangGraph 컴포넌트
import pymysql  # MySQL 데이터베이스 연동

# 시각화 관련
import plotly.graph_objects as go  # 3D 그래프 생성
import plotly.express as px  # 간편한 시각화
import numpy as np  # 수치 연산
import ipywidgets as widgets  # 노트북 위젯
from IPython.display import display  # 노트북 출력
import datetime  # 시간 처리

# ================================================================
# 1. 상태 정의 (AgentState)
# ================================================================
# - 에이전트 간에 전달되는 상태를 정의하는 자료 구조
# - query: 사용자 입력
# - keywords: 추출된 검색 키워드
# - search_results: 검색된 문서 목록 (파일명, 위치, 요약, 관련성 등)
# - context: 검색된 문서들의 결합 내용
# - result: 최종 생성된 답변
# ================================================================
class AgentState(TypedDict):
    query: str
    keywords: str
    search_results: List[Dict[str, Any]]
    context: str
    result: str

# ================================================================
# 2. 데이터베이스 접속 정보
# ================================================================
# - MySQL 데이터베이스 연결을 위한 설정
# - host: localhost
# - user: admin
# - password: 1qazZAQ!
# - db: final
# - charset: utf8mb4 (한글 지원)
# ================================================================
DB_CONFIG = {
    'host': 'localhost',
    'user': 'admin',
    'password': '1qazZAQ!',
    'db': 'final',
    'charset': 'utf8mb4'
}

# ================================================================
# 3. ChromaDB 및 LLM 설정
# ================================================================
# - ChromaDB: 벡터 데이터베이스로 문서 임베딩 저장
# - OllamaEmbeddings: 문서 임베딩용 Ollama 모델 (exaone3.5:2.4b)
# - Ollama: 텍스트 생성용 Ollama 모델 (exaone3.5:2.4b)
# - ChatOllama: 챗봇용 Ollama 모델 (exaone3.5:2.4b, temperature=0.1)
# ================================================================
CHROMA_PATH = "./rag_chroma/documents/title_summary/"  # ChromaDB 저장 경로
EMBEDDINGS = OllamaEmbeddings(model="exaone3.5:2.4b")  # 임베딩 모델
LLM = Ollama(model="exaone3.5:2.4b")  # LLM
CHAT_LLM = ChatOllama(model="exaone3.5:2.4b", temperature=0.1)  # 챗 모델

# ================================================================
# 4. 노트북용 3D 시각화 클래스 (RAGNotebookVisualizer)
# ================================================================
# - RAG 처리 과정을 3D로 시각화하는 클래스
# - 에이전트 상태, 검색 결과, 진행률을 실시간으로 보여줌
# ================================================================
class RAGNotebookVisualizer:
    def __init__(self):
        # 에이전트의 3D 공간 위치 (x, y, z)
        self.agent_positions = {
            'extractor': (0.0, 0.0, 0.0),       # 키워드 추출
            'rag_search': (2.0, 0.0, 1.0),      # 문서 검색
            'answer_generator': (4.0, 0.0, 2.0), # 답변 생성
            'result_formatter': (6.0, 0.0, 1.0)  # 결과 형식화
        }
        self.agent_names = list(self.agent_positions.keys())
        # 에이전트 초기 상태 (대기중)
        self.agent_status = {name: 'waiting' for name in self.agent_names}

        # 3D FigureWidget 생성
        self.fig3d = go.FigureWidget()
        self._init_3d_scene()  # 3D 장면 초기화

        # 관련성 막대 그래프 (검색 결과)
        self.bar_fig = go.FigureWidget(
            data=[go.Bar(x=[], y=[], text=[], textposition='auto')],
            layout=go.Layout(
                title='검색 결과 관련성 (%)',
                height=260,
                margin=dict(t=40)
            )
        )

        # 진행률 바 (에이전트 진행 상황)
        self.progress = widgets.FloatProgress(
            value=0.0,
            min=0.0,
            max=100.0,
            description='진행률:',
            bar_style='info'
        )

        # 로그 텍스트 영역 (처리 기록)
        self.log = widgets.Textarea(
            value='',
            placeholder='로그가 여기에 표시됩니다.',
            layout=widgets.Layout(width='100%', height='600px')
        )

        # 전체 UI 레이아웃 (3D + 차트 + 로그)
        left_panel = widgets.VBox([self.fig3d], layout=widgets.Layout(width='60%'))
        right_panel = widgets.VBox([self.bar_fig, self.progress, self.log], layout=widgets.Layout(width='40%'))
        self.container = widgets.HBox([left_panel, right_panel])

    def _init_3d_scene(self):
        """3D 장면을 초기화하고 에이전트 노드/연결선을 추가"""
        # 에이전트 위치 (x, y, z)
        xs = [pos[0] for pos in self.agent_positions.values()]
        ys = [pos[1] for pos in self.agent_positions.values()]
        zs = [pos[2] for pos in self.agent_positions.values()]

        # 에이전트 노드 (마커 + 텍스트)
        node_trace = go.Scatter3d(
            x=xs,
            y=ys,
            z=zs,
            mode='markers+text',  # 마커와 텍스트 함께 표시
            marker=dict(
                size=14,            # 마커 크기
                color=['gray'] * len(self.agent_names),  # 초기 색상 (회색)
                opacity=0.9,         # 투명도
                line=dict(width=2, color='DarkSlateGrey')  # 마커 테두리
            ),
            text=self.agent_names,   # 노드 텍스트
            textposition="top center",  # 텍스트 위치
            textfont=dict(size=12, color='black'),
            name="에이전트"
        )
        self.fig3d.add_trace(node_trace)

        # 에이전트 간 연결선 (에지)
        for i in range(len(self.agent_names) - 1):
            a = self.agent_positions[self.agent_names[i]]
            b = self.agent_positions[self.agent_names[i+1]]
            edge = go.Scatter3d(
                x=[a[0], b[0]],
                y=[a[1], b[1]],
                z=[a[2], b[2]],
                mode='lines',
                line=dict(color='lightgray', width=4),
                showlegend=False
            )
            self.fig3d.add_trace(edge)

        # 검색 결과 placeholder (업데이트 예정)
        search_trace = go.Scatter3d(
            x=[], y=[], z=[],
            mode='markers+text',
            marker=dict(
                size=[], 
                color=[], 
                colorscale='Viridis',  # 색상 척도
                cmin=0, 
                cmax=100, 
                opacity=0.8, 
                showscale=True,       # 색상 바 표시
                colorbar=dict(title='관련성 (%)')
            ),
            text=[], 
            textposition='top center',
            name='검색 결과'
        )
        self.fig3d.add_trace(search_trace)

        # 3D 레이아웃 설정
        self.fig3d.update_layout(
            title='🤖 RAG 처리 과정 3D 시각화 (Notebook)',
            scene=dict(
                xaxis=dict(title='처리 단계'),
                yaxis=dict(title='데이터 흐름'),
                zaxis=dict(title='복잡도'),
                camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))  # 3D 뷰 각도
            ),
            height=600,
            margin=dict(l=0, r=0, t=40, b=0),  # 마진
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)  # 범례
        )

    def show(self):
        """시각화 UI를 노트북에 표시"""
        display(self.container)

    def _append_log(self, text: str):
        """로그 메시지를 추가하고 표시"""
        # 현재 시간 포맷: HH:MM:SS
        timestamp = datetime.datetime.now().strftime("%H:%M:%S")
        self.log.value += f"[{timestamp}] {text}\n"

    def update_agent_status(self, agent_name: str, status: str, data: List[Dict[str, Any]] = None):
        """
        에이전트 상태를 업데이트하고 시각화를 갱신
        
        Args:
            agent_name: 업데이트할 에이전트 이름
            status: 'processing', 'completed', 'error', 'waiting'
            data: 검색 결과 데이터 (rag_search 에이전트 전용)
        """
        # 1. 로그에 상태 추가
        status_text = status.capitalize()
        self._append_log(f"{agent_name} -> {status_text}")

        # 2. 에이전트 상태 저장
        if agent_name in self.agent_status:
            self.agent_status[agent_name] = status

        # 3. 색상 매핑: 상태에 따른 색상
        def get_status_color(s):
            if s == 'processing': return 'yellow'
            elif s == 'completed': return 'green'
            elif s == 'error': return 'red'
            else: return 'gray'  # waiting

        # 4. 3D 노드 색상 업데이트
        colors = [get_status_color(self.agent_status[name]) for name in self.agent_names]
        self.fig3d.data[0].marker.color = colors

        # 5. 진행률 업데이트 (완료된 에이전트 비율)
        completed_count = sum(1 for s in self.agent_status.values() if s == 'completed')
        progress_percent = (completed_count / len(self.agent_status)) * 100
        self.progress.value = progress_percent

        # 6. 검색 결과가 있는 경우 시각화 업데이트
        if data and agent_name == 'rag_search':
            self.update_search_results(data)

    def update_search_results(self, results: List[Dict[str, Any]]):
        """검색 결과를 3D 그래프와 막대 차트에 업데이트
        
        Args:
            results: 검색 결과 목록 (relevance, file_name, ...)
        """
        # 1. 관련성 순으로 내림차순 정렬
        results_sorted = sorted(results, key=lambda x: x.get('relevance', 0), reverse=True)
        
        # 2. 3D 검색 결과 업데이트
        base_x, base_y, base_z = self.agent_positions['rag_search']  # rag_search 위치
        n = len(results_sorted)
        xs, ys, zs, sizes, colors, texts = [], [], [], [], [], []
        radius = 1.5  # 원형 배치 반지름

        # 3. 검색 결과를 원형으로 배치
        for i, res in enumerate(results_sorted):
            angle = (i * 2 * np.pi) / n  # 각도
            # 3D 좌표: rag_search 위치를 중심으로 원형 배치
            x = base_x + radius * np.cos(angle)
            y = base_y + radius * np.sin(angle)
            z = base_z + (res.get('relevance', 0) / 100) * 2  # 관련성에 따른 높이
            
            xs.append(x)
            ys.append(y)
            zs.append(z)
            sizes.append(max(6, res.get('relevance', 0) / 4))  # 관련성에 따른 크기
            colors.append(res.get('relevance', 0))  # 색상 (관련성)
            # 텍스트: 파일명 + 관련성
            text = f"{res.get('file_name', '파일')}<br>{res.get('relevance', 0)}%"
            texts.append(text)

        # 4. 3D 검색 결과 trace 업데이트
        for trace in self.fig3d.data:
            if trace.name == '검색 결과':
                trace.x = xs
                trace.y = ys
                trace.z = zs
                trace.marker.size = sizes
                trace.marker.color = colors
                trace.text = texts
                break

        # 5. 막대 차트 업데이트 (관련성)
        file_names = [res.get('file_name', '') for res in results_sorted]
        relevances = [res.get('relevance', 0) for res in results_sorted]
        self.bar_fig.data[0].x = file_names
        self.bar_fig.data[0].y = relevances
        self.bar_fig.data[0].text = [f"{r}%" for r in relevances]
        self.bar_fig.data[0].marker.color = px.colors.sequential.Viridis  # 색상 척도

        # 6. 막대 차트 레이아웃 업데이트 (너비 조절)
        self.bar_fig.update_layout(
            xaxis_tickangle=-45,  # 파일명 45도 회전
            margin=dict(b=100)    # 파일명 간격 확보
        )


# ================================================================
# 5. 시각화 객체 생성
# ================================================================
# - 위에서 정의한 RAGNotebookVisualizer 인스턴스 생성
# - 노트북에서 show() 메소드를 호출해 UI 표시
# ================================================================
visualizer = RAGNotebookVisualizer()

# ================================================================
# 6. RAG 에이전트 함수
# ================================================================
# - 4개의 에이전트: 키워드 추출, RAG 검색, 답변 생성, 결과 형식화
# - 각 에이전트는 visualizer.update_agent_status()로 상태를 업데이트
# ================================================================

# 6.1. 키워드 추출 에이전트
def extractor_agent(state: AgentState):
    # 상태: processing
    visualizer.update_agent_status('extractor', 'processing')
    
    # 프롬프트 템플릿: 사용자 질의에서 키워드 추출
    keyword_prompt = PromptTemplate.from_template(
        """사용자의 질문에서 띄어쓰기 확인하고 찾고자 하는 키워드를 쉼표로 구분하여 출력하세요.
        벡터스토어 검색을 위한 최적의 키워드를 추출해주세요.
        \n질문: {query}"""
    )
    # 프롬프트 포맷팅
    formatted_prompt = keyword_prompt.format(query=state["query"])
    # LLM 호출
    keywords = LLM.invoke(formatted_prompt)
    
    # 상태: completed
    visualizer.update_agent_status('extractor', 'completed')
    # state 업데이트 (keywords 추가)
    return {**state, "keywords": keywords.strip()}

# 6.2. RAG 검색 에이전트
def rag_search_agent(state: AgentState):
    # 상태: processing
    visualizer.update_agent_status('rag_search', 'processing')
    
    try:
        # 1. ChromaDB 로드
        vectorstore = Chroma(persist_directory=CHROMA_PATH, embedding_function=EMBEDDINGS)
        # 2. 벡터스토어 검색 (k=10: 상위 10개)
        results = vectorstore.similarity_search_with_score(state["keywords"], k=10)

        if not results:
            # 검색 결과 없음
            visualizer.update_agent_status('rag_search', 'completed', [])
            return {**state, "search_results": [], "context": ""}

        # 3. 문서 중복 제거 (최고 유사도 유지)
        best_doc_info = {}
        for doc, score in results:
            doc_id = doc.metadata.get("id")  # 문서 ID
            if doc_id:
                content = doc.page_content.strip()
                if not content:  # 내용이 빈 경우 스킵
                    continue
                # 동일 문서 ID 중 가장 높은 유사도를 유지
                if doc_id not in best_doc_info or score < best_doc_info[doc_id][0]:
                    best_doc_info[doc_id] = (score, content)

        if not best_doc_info:
            visualizer.update_agent_status('rag_search', 'completed', [])
            return {**state, "search_results": [], "context": ""}

        # 4. 유사도 정규화 (0-100%로 변환)
        scores = [score for score, _ in best_doc_info.values()]
        min_score = min(scores)
        max_score = max(scores)

        # 5. MySQL에서 파일 메타데이터 조회
        conn = None
        search_results = []
        try:
            conn = pymysql.connect(**DB_CONFIG)
            cursor = conn.cursor(pymysql.cursors.DictCursor)

            for doc_id, (score, content) in best_doc_info.items():
                # MySQL: documents 테이블에서 문서 ID로 조회
                cursor.execute("SELECT * FROM documents WHERE id = %s", (doc_id,))
                row = cursor.fetchone()
                if not row:
                    continue  # 데이터 없을 경우 스킵

                # 6. 관련성 % 계산
                if min_score == max_score:
                    relevance = 100.0  # 모든 문서 동일 유사도
                else:
                    # min-max normalization: 0-100%로 변환
                    relevance = round((1 - (score - min_score) / (max_score - min_score)) * 100, 1)

                # 7. 검색 결과 목록에 추가
                search_results.append({
                    "relevance": relevance,
                    "file_name": row["file_name"],
                    "file_location": row["file_location"],
                    "summary": (row["summary"][:200] + "...") if row["summary"] else "요약 없음",
                    "title": row["title"],
                    "created_at": row["created_at"].strftime("%Y-%m-%d") if row["created_at"] else "N/A",
                    "doc_type": row["doc_type"],
                    "content": content
                })

            # 8. 관련성 순으로 정렬 (내림차순) 및 상위 10개
            search_results.sort(key=lambda x: x["relevance"], reverse=True)
            search_results = search_results[:10]
            # 9. 문서 내용 결합 (context)
            context = "\n\n".join([res['content'] for res in search_results])

        except Exception as e:
            # MySQL 오류 처리
            print(f"❌ MySQL 오류: {e}")
            visualizer.update_agent_status('rag_search', 'error')
            return {**state, "search_results": [], "context": ""}
        finally:
            if conn:
                conn.close()

        # 상태: completed + 검색 결과 데이터 전달
        visualizer.update_agent_status('rag_search', 'completed', search_results)
        return {**state, "search_results": search_results, "context": context}

    except Exception as e:
        # 일반 오류 처리
        print(f"❌ RAG 검색 오류: {e}")
        visualizer.update_agent_status('rag_search', 'error')
        return {**state, "search_results": [], "context": ""}

# 6.3. 답변 생성 에이전트
def answer_generator_agent(state: AgentState):
    # 상태: processing
    visualizer.update_agent_status('answer_generator', 'processing')
    
    # 검색 결과가 없는 경우
    if not state["search_results"]:
        visualizer.update_agent_status('answer_generator', 'completed')
        return {**state, "result": "관련 정보를 찾을 수 없습니다."}

    try:
        # 1. 프롬프트 템플릿
        prompt = ChatPromptTemplate.from_template(
            """다음 문서 내용들을 바탕으로 사용자 질문에 답변해 주세요.
            - 문서에 직접 언급된 내용만을 바탕으로 답변해야 합니다.
            - 정보가 없는 경우 "정보가 없습니다"라고 명확히 답변해 주세요.
            - 항상 공손하고 전문적인 어조를 유지해 주세요.

            ### 검색 결과 요약:
            {search_summary}

            ### 문서 내용:
            {context}

            ### 사용자 질문:
            {query}

            ### 답변:
            찾은 문서중에 몇번째 정보가 가장 관련이 높은지를 상세히 답변해 주세요.
            (몇번째, 무슨파일인지, 어디에 있는지, 요약은 무엇인지 등)
            """
        )

        # 2. 검색 결과 요약 생성
        search_summary = "\n".join([
            f"{i+1}순위 ({res['relevance']}%): {res['file_name']} ({res['doc_type']}) - {res['summary']}"
            for i, res in enumerate(state["search_results"])
        ])

        # 3. LLM 체인: 프롬프트 + LLM + 출력 파서
        chain = prompt | CHAT_LLM | StrOutputParser()
        result = chain.invoke({
            "search_summary": search_summary,
            "context": state["context"],
            "query": state["query"]
        })

        # 상태: completed
        visualizer.update_agent_status('answer_generator', 'completed')
        return {**state, "result": result.strip()}

    except Exception as e:
        print(f"❌ 답변 생성 오류: {e}")
        visualizer.update_agent_status('answer_generator', 'error')
        return {**state, "result": "답변을 생성하는 중에 오류가 발생했습니다."}

# 6.4. 결과 형식화 에이전트
def result_formatter_agent(state: AgentState):
    # 상태: processing
    visualizer.update_agent_status('result_formatter', 'processing')
    
    # 검색 결과가 없는 경우
    if not state["search_results"]:
        visualizer.update_agent_status('result_formatter', 'completed')
        return state

    try:
        # 1. 검색 결과 형식화 (Markdown 스타일)
        formatted_result = "\n🔍 검색 결과:\n"
        for i, res in enumerate(state["search_results"], 1):
            formatted_result += f"\n--- {i}순위 ({res['relevance']}%) ---\n"
            formatted_result += f"📄 파일명: {res['file_name']}\n"
            formatted_result += f"📁 위치: {res['file_location']}\n"
            formatted_result += f"📝 요약: {res['summary']}\n"
            formatted_result += f"🏷️ 유형: {res['doc_type']}\n"
            formatted_result += f"🕒 등록일: {res['created_at']}\n"

        formatted_result += f"\n💬 AI 답변:\n{state['result']}"

        # 상태: completed
        visualizer.update_agent_status('result_formatter', 'completed')
        return {**state, "result": formatted_result}

    except Exception as e:
        print(f"❌ 결과 형식화 오류: {e}")
        visualizer.update_agent_status('result_formatter', 'error')
        return state

# ================================================================
# 7. LangGraph 구성
# ================================================================
# - 에이전트들을 노드로 연결한 그래프 생성
# - 실행 순서: extractor → rag_search → answer_generator → result_formatter
# ================================================================
graph = StateGraph(AgentState)  # AgentState를 상태로 사용하는 그래프
graph.add_node("extractor", extractor_agent)  # 노드 추가
graph.add_node("rag_search", rag_search_agent)
graph.add_node("answer_generator", answer_generator_agent)
graph.add_node("result_formatter", result_formatter_agent)

# 시작 노드 설정
graph.set_entry_point("extractor")

# 노드 연결
graph.add_edge("extractor", "rag_search")
graph.add_edge("rag_search", "answer_generator")
graph.add_edge("answer_generator", "result_formatter")
graph.add_edge("result_formatter", END)  # END: 그래프 종료

# 그래프 컴파일
app = graph.compile()

# ================================================================
# 8. 실행 예시 (Jupyter Notebook)
# ================================================================
# 1) 시각화 UI 표시
visualizer.show()

# 2) RAG 파이프라인 실행
state = {
    "query": "기초생활수급자 차상위계층",  # 사용자 질의
    "keywords": "",            # 초기 키워드
    "search_results": [],      # 검색 결과
    "context": "",             # 문서 내용
    "result": ""               # 최종 답변
}

# 3) invoke()로 그래프 실행
result = app.invoke(state)

# 4) 최종 결과 출력
print("\n" + "="*50)
print("📊 RAG 처리 완료! 최종 결과:")
print("="*50)
print(result["result"])

# (visualizer는 에이전트 실행 도중 실시간으로 업데이트됩니다.)

    'data': [{'marker': {'color': [gray, gray, gray, gray],
     …


📊 RAG 처리 완료! 최종 결과:

🔍 검색 결과:

--- 1순위 (100.0%) ---
📄 파일명: 2025년 신문 구독 지원 신청 전 필독사항.pdf
📁 위치: fileList/2025년 신문 구독 지원 신청 전 필독사항.pdf
📝 요약: 정부는 취약계층의 정보 격차 해소를 위해 기초생활수급자, 차상위계층, 장애인을 대상으로 매년 1년간 무료 신문 구독 지원 사업을 진행한다. 신청 기간은 2024년 11월 13일부터 11월 27일까지로, 접수 시 거주지 인근 신문 지국의 배달 가능 여부를 확인하고 신청해야 한다. 1명의 가족당 한 부를 지원하며, 선정 결과는 2024년 12월에 발표되고, 신...
🏷️ 유형: .pdf
🕒 등록일: 2025-10-21

--- 2순위 (61.2%) ---
📄 파일명: 세계기사.docx
📁 위치: fileList/세계기사.docx
📝 요약: 중국 상하이 출신의 금융학 석사 출신 청년 자오덴은 과도한 가족 압박과 외로움으로 인해 뉴질랜드 이주 후 다양한 도시에서 생활하며 학문적 성공을 이루었으나, 현재는 한 달에 약 2만원으로 생활하는 노숙자로 변모했다. 그는 깊이 느끼는 외로움과 부모와의 갈등 속에서 극단적인 저비용 생활을 선택해 자유를 추구하며, 의미 있는 활동으로 삶의 가치를 찾고 있다. ...
🏷️ 유형: .docx
🕒 등록일: 2025-10-21

--- 3순위 (40.4%) ---
📄 파일명: 소름돋는정책원해.docx
📁 위치: fileList/소름돋는정책원해.docx
📝 요약: 이재명 정부는 청년주간을 선언하며 '국민주권정부 청년정책 추진방향'을 발표, 청년의 체감과 참여를 핵심으로 삼아 일자리 창출, 주거 지원 확대, 실질적 정책 참여 확대에 중점을 둔 139개 세부과제를 제시했다. 특히 청년 월세지원 사업의 확대와 대기업 신규 채용 확대를 통해 청년들의 일자리 문제를 해결하려는 노력을 보여주며, 이는 청년 세대의 가장 큰 어려...
🏷️ 유형: .docx
🕒 등록일: 2025-10-21

--- 4순위 (3

# 색상 변경

In [59]:
# ================================================================
# 노트북용 RAG + 3D 시각화 통합 코드
# ================================================================
# 이 코드는 RAG(Retrieval-Augmented Generation) 시스템을 Jupyter Notebook 환경에서 
# 실행하고, 처리 과정을 3D 시각화하는 통합 솔루션입니다.
# - 사용자가 입력한 질의에 대해 관련 문서를 검색하고, AI가 답변을 생성하는 과정을
#   실시간으로 3D 그래프와 차트로 모니터링할 수 있습니다.
# - 시스템은 4개의 에이전트로 구성되어 있으며, LangGraph로 연결되어 순차적으로 실행됩니다.
# ================================================================

from typing import TypedDict, List, Dict, Any
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate  # 프롬프트 템플릿 생성
from langchain_core.output_parsers import StrOutputParser  # LLM 출력 파싱
from langchain_community.llms import Ollama  # Ollama LLM
from langchain_community.chat_models import ChatOllama  # Ollama 챗 모델
from langchain_community.vectorstores import Chroma  # 벡터 데이터베이스
from langchain_community.embeddings import OllamaEmbeddings  # Ollama 임베딩
from langgraph.graph import StateGraph, END  # LangGraph 컴포넌트
import pymysql  # MySQL 데이터베이스 연동

# 시각화 관련
import plotly.graph_objects as go  # 3D 그래프 생성
import plotly.express as px  # 간편한 시각화
import numpy as np  # 수치 연산
import ipywidgets as widgets  # 노트북 위젯
from IPython.display import display  # 노트북 출력
import datetime  # 시간 처리

# ================================================================
# 1. 상태 정의 (AgentState)
# ================================================================
# - 에이전트 간에 전달되는 상태를 정의하는 자료 구조
# - query: 사용자 입력
# - keywords: 추출된 검색 키워드
# - search_results: 검색된 문서 목록 (파일명, 위치, 요약, 관련성 등)
# - context: 검색된 문서들의 결합 내용
# - result: 최종 생성된 답변
# ================================================================
class AgentState(TypedDict):
    query: str
    keywords: str
    search_results: List[Dict[str, Any]]
    context: str
    result: str

# ================================================================
# 2. 데이터베이스 접속 정보
# ================================================================
# - MySQL 데이터베이스 연결을 위한 설정
# - host: localhost
# - user: admin
# - password: 1qazZAQ!
# - db: final
# - charset: utf8mb4 (한글 지원)
# ================================================================
DB_CONFIG = {
    'host': 'localhost',
    'user': 'admin',
    'password': '1qazZAQ!',
    'db': 'final',
    'charset': 'utf8mb4'
}

# ================================================================
# 3. ChromaDB 및 LLM 설정
# ================================================================
# - ChromaDB: 벡터 데이터베이스로 문서 임베딩 저장
# - OllamaEmbeddings: 문서 임베딩용 Ollama 모델 (exaone3.5:2.4b)
# - Ollama: 텍스트 생성용 Ollama 모델 (exaone3.5:2.4b)
# - ChatOllama: 챗봇용 Ollama 모델 (exaone3.5:2.4b, temperature=0.1)
# ================================================================
CHROMA_PATH = "./rag_chroma/documents/title_summary/"  # ChromaDB 저장 경로
EMBEDDINGS = OllamaEmbeddings(model="exaone3.5:2.4b")  # 임베딩 모델
LLM = Ollama(model="exaone3.5:2.4b")  # LLM
CHAT_LLM = ChatOllama(model="exaone3.5:2.4b", temperature=0.1)  # 챗 모델

# ================================================================
# 4. 노트북용 3D 시각화 클래스 (RAGNotebookVisualizer)
# ================================================================
# - RAG 처리 과정을 3D로 시각화하는 클래스
# - 에이전트 상태, 검색 결과, 진행률을 실시간으로 보여줌
# ================================================================
class RAGNotebookVisualizer:
    def __init__(self):
        # 에이전트의 3D 공간 위치 (x, y, z)
        self.agent_positions = {
            'extractor': (0.0, 0.0, 0.0),       # 키워드 추출
            'rag_search': (2.0, 0.0, 1.0),      # 문서 검색
            'answer_generator': (4.0, 0.0, 2.0), # 답변 생성
            'result_formatter': (6.0, 0.0, 1.0)  # 결과 형식화
        }
        self.agent_names = list(self.agent_positions.keys())
        # 에이전트 초기 상태 (대기중)
        self.agent_status = {name: 'waiting' for name in self.agent_names}

        # 3D FigureWidget 생성
        self.fig3d = go.FigureWidget()
        self._init_3d_scene()  # 3D 장면 초기화

        # 관련성 막대 그래프 (검색 결과)
        self.bar_fig = go.FigureWidget(
            data=[go.Bar(x=[], y=[], text=[], textposition='auto')],
            layout=go.Layout(
                title='검색 결과 관련성 (%)',
                height=260,
                margin=dict(t=40)
            )
        )

        # 진행률 바 (에이전트 진행 상황)
        self.progress = widgets.FloatProgress(
            value=0.0,
            min=0.0,
            max=100.0,
            description='진행률:',
            bar_style='info'
        )

        # 로그 텍스트 영역 (처리 기록)
        self.log = widgets.Textarea(
            value='',
            placeholder='로그가 여기에 표시됩니다.',
            layout=widgets.Layout(width='100%', height='600px')
        )

        # 전체 UI 레이아웃 (3D + 차트 + 로그)
        left_panel = widgets.VBox([self.fig3d], layout=widgets.Layout(width='60%'))
        right_panel = widgets.VBox([self.bar_fig, self.progress, self.log], layout=widgets.Layout(width='40%'))
        self.container = widgets.HBox([left_panel, right_panel])

    def _init_3d_scene(self):
        """3D 장면을 초기화하고 에이전트 노드/연결선을 추가"""
        # 에이전트 위치 (x, y, z)
        xs = [pos[0] for pos in self.agent_positions.values()]
        ys = [pos[1] for pos in self.agent_positions.values()]
        zs = [pos[2] for pos in self.agent_positions.values()]

        # 에이전트 노드 (마커 + 텍스트)
        node_trace = go.Scatter3d(
            x=xs,
            y=ys,
            z=zs,
            mode='markers+text',  # 마커와 텍스트 함께 표시
            marker=dict(
                size=14,            # 마커 크기
                color=['gray'] * len(self.agent_names),  # 초기 색상 (회색)
                opacity=0.9,         # 투명도
                line=dict(width=2, color='DarkSlateGrey')  # 마커 테두리
            ),
            text=self.agent_names,   # 노드 텍스트
            textposition="top center",  # 텍스트 위치
            textfont=dict(size=12, color='black'),
            name="에이전트"
        )
        self.fig3d.add_trace(node_trace)

        # 에이전트 간 연결선 (에지)
        for i in range(len(self.agent_names) - 1):
            a = self.agent_positions[self.agent_names[i]]
            b = self.agent_positions[self.agent_names[i+1]]
            edge = go.Scatter3d(
                x=[a[0], b[0]],
                y=[a[1], b[1]],
                z=[a[2], b[2]],
                mode='lines',
                line=dict(color='lightgray', width=4),
                showlegend=False
            )
            self.fig3d.add_trace(edge)

        # 검색 결과 placeholder (업데이트 예정)
        search_trace = go.Scatter3d(
            x=[], y=[], z=[],
            mode='markers+text',
            marker=dict(
                size=[], 
                color=[], 
                colorscale='Viridis',  # 색상 척도
                cmin=0, 
                cmax=100, 
                opacity=0.8, 
                showscale=True,       # 색상 바 표시
                colorbar=dict(title='관련성 (%)')
            ),
            text=[], 
            textposition='top center',
            name='검색 결과'
        )
        self.fig3d.add_trace(search_trace)

        # 3D 레이아웃 설정
        self.fig3d.update_layout(
            title='🤖 RAG 처리 과정 3D 시각화 (Notebook)',
            scene=dict(
               	xaxis=dict(title='⏩ 진행 단계'),    # 기존: 처리 단계
                yaxis=dict(title='🗂️ 작업 공간'),    # 기존: 데이터 흐름
                zaxis=dict(title='⭐ 중요도(관련성)'), # 기존: 복잡도
                camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
            ),
            height=600,
            margin=dict(l=0, r=0, t=40, b=0),  # 마진
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)  # 범례
        )

    def show(self):
        """시각화 UI를 노트북에 표시"""
        display(self.container)

    def _append_log(self, text: str):
        """로그 메시지를 추가하고 표시"""
        # 현재 시간 포맷: HH:MM:SS
        timestamp = datetime.datetime.now().strftime("%H:%M:%S")
        self.log.value += f"[{timestamp}] {text}\n"

    def update_agent_status(self, agent_name: str, status: str, data: List[Dict[str, Any]] = None):
        """
        에이전트 상태를 업데이트하고 시각화를 갱신
        
        Args:
            agent_name: 업데이트할 에이전트 이름
            status: 'processing', 'completed', 'error', 'waiting'
            data: 검색 결과 데이터 (rag_search 에이전트 전용)
        """
        # 1. 로그에 상태 추가
        status_text = status.capitalize()
        self._append_log(f"{agent_name} -> {status_text}")

        # 2. 에이전트 상태 저장
        if agent_name in self.agent_status:
            self.agent_status[agent_name] = status

        # 3. 색상 매핑: 상태에 따른 색상
        def get_status_color(s):
            if s == 'processing': return 'yellow'
            elif s == 'completed': return 'green'
            elif s == 'error': return 'red'
            else: return 'gray'  # waiting

        # 4. 3D 노드 색상 업데이트
        colors = [get_status_color(self.agent_status[name]) for name in self.agent_names]
        self.fig3d.data[0].marker.color = colors

        # 5. 진행률 업데이트 (완료된 에이전트 비율)
        completed_count = sum(1 for s in self.agent_status.values() if s == 'completed')
        progress_percent = (completed_count / len(self.agent_status)) * 100
        self.progress.value = progress_percent

        # 6. 검색 결과가 있는 경우 시각화 업데이트
        if data and agent_name == 'rag_search':
            self.update_search_results(data)

    def update_search_results(self, results: List[Dict[str, Any]]):
        """검색 결과를 3D 그래프와 막대 차트에 업데이트
        
        Args:
            results: 검색 결과 목록 (relevance, file_name, ...)
        """
        # 1. 관련성 순으로 내림차순 정렬
        results_sorted = sorted(results, key=lambda x: x.get('relevance', 0), reverse=True)
        
        # 2. 3D 검색 결과 업데이트
        base_x, base_y, base_z = self.agent_positions['rag_search']  # rag_search 위치
        n = len(results_sorted)
        xs, ys, zs, sizes, colors, texts = [], [], [], [], [], []
        radius = 1.5  # 원형 배치 반지름

        # 3. 검색 결과를 원형으로 배치
        for i, res in enumerate(results_sorted):
            angle = (i * 2 * np.pi) / n  # 각도
            # 3D 좌표: rag_search 위치를 중심으로 원형 배치
            x = base_x + radius * np.cos(angle)
            y = base_y + radius * np.sin(angle)
            z = base_z + (res.get('relevance', 0) / 100) * 2  # 관련성에 따른 높이
            
            xs.append(x)
            ys.append(y)
            zs.append(z)
            sizes.append(max(6, res.get('relevance', 0) / 4))  # 관련성에 따른 크기
            colors.append(res.get('relevance', 0))  # 색상 (관련성)
            # 텍스트: 파일명 + 관련성
            text = f"{res.get('file_name', '파일')}<br>{res.get('relevance', 0)}%"
            texts.append(text)

        # 4. 3D 검색 결과 trace 업데이트
        for trace in self.fig3d.data:
            if trace.name == '검색 결과':
                trace.x = xs
                trace.y = ys
                trace.z = zs
                trace.marker.size = sizes
                trace.marker.color = colors
                trace.text = texts
                break

        # 5. 막대 차트 업데이트 (관련성)
        file_names = [res.get('file_name', '') for res in results_sorted]
        relevances = [res.get('relevance', 0) for res in results_sorted]

        # FigureWidget의 실시간 업데이트를 위해 batch_update 사용 권장
        with self.bar_fig.batch_update():
            self.bar_fig.data[0].x = file_names
            self.bar_fig.data[0].y = relevances
            self.bar_fig.data[0].text = [f"{r}%" for r in relevances]
            
            # 핵심 수정: 색상을 데이터 값(relevances)에 매핑
            self.bar_fig.data[0].marker.color = relevances
            
            # 'Viridis' 스케일: 낮은 값(보라색/어두움) -> 높은 값(노란색/밝음)
            self.bar_fig.data[0].marker.colorscale = 'Viridis'
            
            # 색상 범위 고정 (0~100%) - 선택 사항이지만 일관된 색상 표현에 도움
            self.bar_fig.data[0].marker.cmin = 0
            self.bar_fig.data[0].marker.cmax = 100
            self.bar_fig.data[0].marker.showscale = True # 우측에 색상바 표시

        # 6. 막대 차트 레이아웃 업데이트 (너비 조절)
        self.bar_fig.update_layout(
            xaxis_tickangle=-45,  # 파일명 45도 회전
            margin=dict(b=100)    # 파일명 간격 확보
        )


# ================================================================
# 5. 시각화 객체 생성
# ================================================================
# - 위에서 정의한 RAGNotebookVisualizer 인스턴스 생성
# - 노트북에서 show() 메소드를 호출해 UI 표시
# ================================================================
visualizer = RAGNotebookVisualizer()

# ================================================================
# 6. RAG 에이전트 함수
# ================================================================
# - 4개의 에이전트: 키워드 추출, RAG 검색, 답변 생성, 결과 형식화
# - 각 에이전트는 visualizer.update_agent_status()로 상태를 업데이트
# ================================================================

# 6.1. 키워드 추출 에이전트
def extractor_agent(state: AgentState):
    # 상태: processing
    visualizer.update_agent_status('extractor', 'processing')
    
    # 프롬프트 템플릿: 사용자 질의에서 키워드 추출
    keyword_prompt = PromptTemplate.from_template(
        """사용자의 질문에서 띄어쓰기 확인하고 찾고자 하는 키워드를 쉼표로 구분하여 출력하세요.
        벡터스토어 검색을 위한 최적의 키워드를 추출해주세요.
        \n질문: {query}"""
    )
    # 프롬프트 포맷팅
    formatted_prompt = keyword_prompt.format(query=state["query"])
    # LLM 호출
    keywords = LLM.invoke(formatted_prompt)
    
    # 상태: completed
    visualizer.update_agent_status('extractor', 'completed')
    # state 업데이트 (keywords 추가)
    return {**state, "keywords": keywords.strip()}

# 6.2. RAG 검색 에이전트
def rag_search_agent(state: AgentState):
    # 상태: processing
    visualizer.update_agent_status('rag_search', 'processing')
    
    try:
        # 1. ChromaDB 로드
        vectorstore = Chroma(persist_directory=CHROMA_PATH, embedding_function=EMBEDDINGS)
        # 2. 벡터스토어 검색 (k=10: 상위 10개)
        results = vectorstore.similarity_search_with_score(state["keywords"], k=10)

        if not results:
            # 검색 결과 없음
            visualizer.update_agent_status('rag_search', 'completed', [])
            return {**state, "search_results": [], "context": ""}

        # 3. 문서 중복 제거 (최고 유사도 유지)
        best_doc_info = {}
        for doc, score in results:
            doc_id = doc.metadata.get("id")  # 문서 ID
            if doc_id:
                content = doc.page_content.strip()
                if not content:  # 내용이 빈 경우 스킵
                    continue
                # 동일 문서 ID 중 가장 높은 유사도를 유지
                if doc_id not in best_doc_info or score < best_doc_info[doc_id][0]:
                    best_doc_info[doc_id] = (score, content)

        if not best_doc_info:
            visualizer.update_agent_status('rag_search', 'completed', [])
            return {**state, "search_results": [], "context": ""}

        # 4. 유사도 정규화 (0-100%로 변환)
        scores = [score for score, _ in best_doc_info.values()]
        min_score = min(scores)
        max_score = max(scores)

        # 5. MySQL에서 파일 메타데이터 조회
        conn = None
        search_results = []
        try:
            conn = pymysql.connect(**DB_CONFIG)
            cursor = conn.cursor(pymysql.cursors.DictCursor)

            for doc_id, (score, content) in best_doc_info.items():
                # MySQL: documents 테이블에서 문서 ID로 조회
                cursor.execute("SELECT * FROM documents WHERE id = %s", (doc_id,))
                row = cursor.fetchone()
                if not row:
                    continue  # 데이터 없을 경우 스킵

                # 6. 관련성 % 계산
                if min_score == max_score:
                    relevance = 100.0  # 모든 문서 동일 유사도
                else:
                    # min-max normalization: 0-100%로 변환
                    relevance = round((1 - (score - min_score) / (max_score - min_score)) * 100, 1)

                # 7. 검색 결과 목록에 추가
                search_results.append({
                    "relevance": relevance,
                    "file_name": row["file_name"],
                    "file_location": row["file_location"],
                    "summary": (row["summary"][:200] + "...") if row["summary"] else "요약 없음",
                    "title": row["title"],
                    "created_at": row["created_at"].strftime("%Y-%m-%d") if row["created_at"] else "N/A",
                    "doc_type": row["doc_type"],
                    "content": content
                })

            # 8. 관련성 순으로 정렬 (내림차순) 및 상위 10개
            search_results.sort(key=lambda x: x["relevance"], reverse=True)
            search_results = search_results[:10]
            # 9. 문서 내용 결합 (context)
            context = "\n\n".join([res['content'] for res in search_results])

        except Exception as e:
            # MySQL 오류 처리
            print(f"❌ MySQL 오류: {e}")
            visualizer.update_agent_status('rag_search', 'error')
            return {**state, "search_results": [], "context": ""}
        finally:
            if conn:
                conn.close()

        # 상태: completed + 검색 결과 데이터 전달
        visualizer.update_agent_status('rag_search', 'completed', search_results)
        return {**state, "search_results": search_results, "context": context}

    except Exception as e:
        # 일반 오류 처리
        print(f"❌ RAG 검색 오류: {e}")
        visualizer.update_agent_status('rag_search', 'error')
        return {**state, "search_results": [], "context": ""}

# 6.3. 답변 생성 에이전트
def answer_generator_agent(state: AgentState):
    # 상태: processing
    visualizer.update_agent_status('answer_generator', 'processing')
    
    # 검색 결과가 없는 경우
    if not state["search_results"]:
        visualizer.update_agent_status('answer_generator', 'completed')
        return {**state, "result": "관련 정보를 찾을 수 없습니다."}

    try:
        # 1. 프롬프트 템플릿
        prompt = ChatPromptTemplate.from_template(
            """다음 문서 내용들을 바탕으로 사용자 질문에 답변해 주세요.
            - 문서에 직접 언급된 내용만을 바탕으로 답변해야 합니다.
            - 정보가 없는 경우 "정보가 없습니다"라고 명확히 답변해 주세요.
            - 항상 공손하고 전문적인 어조를 유지해 주세요.

            ### 검색 결과 요약:
            {search_summary}

            ### 문서 내용:
            {context}

            ### 사용자 질문:
            {query}

            ### 답변:
            찾은 문서중에 몇번째 정보가 가장 관련이 높은지를 상세히 답변해 주세요.
            (몇번째, 무슨파일인지, 어디에 있는지, 요약은 무엇인지 등)
            """
        )

        # 2. 검색 결과 요약 생성
        search_summary = "\n".join([
            f"{i+1}순위 ({res['relevance']}%): {res['file_name']} ({res['doc_type']}) - {res['summary']}"
            for i, res in enumerate(state["search_results"])
        ])

        # 3. LLM 체인: 프롬프트 + LLM + 출력 파서
        chain = prompt | CHAT_LLM | StrOutputParser()
        result = chain.invoke({
            "search_summary": search_summary,
            "context": state["context"],
            "query": state["query"]
        })

        # 상태: completed
        visualizer.update_agent_status('answer_generator', 'completed')
        return {**state, "result": result.strip()}

    except Exception as e:
        print(f"❌ 답변 생성 오류: {e}")
        visualizer.update_agent_status('answer_generator', 'error')
        return {**state, "result": "답변을 생성하는 중에 오류가 발생했습니다."}

# 6.4. 결과 형식화 에이전트
def result_formatter_agent(state: AgentState):
    # 상태: processing
    visualizer.update_agent_status('result_formatter', 'processing')
    
    # 검색 결과가 없는 경우
    if not state["search_results"]:
        visualizer.update_agent_status('result_formatter', 'completed')
        return state

    try:
        # 1. 검색 결과 형식화 (Markdown 스타일)
        formatted_result = "\n🔍 검색 결과:\n"
        for i, res in enumerate(state["search_results"], 1):
            formatted_result += f"\n--- {i}순위 ({res['relevance']}%) ---\n"
            formatted_result += f"📄 파일명: {res['file_name']}\n"
            formatted_result += f"📁 위치: {res['file_location']}\n"
            formatted_result += f"📝 요약: {res['summary']}\n"
            formatted_result += f"🏷️ 유형: {res['doc_type']}\n"
            formatted_result += f"🕒 등록일: {res['created_at']}\n"

        formatted_result += f"\n💬 AI 답변:\n{state['result']}"

        # 상태: completed
        visualizer.update_agent_status('result_formatter', 'completed')
        return {**state, "result": formatted_result}

    except Exception as e:
        print(f"❌ 결과 형식화 오류: {e}")
        visualizer.update_agent_status('result_formatter', 'error')
        return state

# ================================================================
# 7. LangGraph 구성
# ================================================================
# - 에이전트들을 노드로 연결한 그래프 생성
# - 실행 순서: extractor → rag_search → answer_generator → result_formatter
# ================================================================
graph = StateGraph(AgentState)  # AgentState를 상태로 사용하는 그래프
graph.add_node("extractor", extractor_agent)  # 노드 추가
graph.add_node("rag_search", rag_search_agent)
graph.add_node("answer_generator", answer_generator_agent)
graph.add_node("result_formatter", result_formatter_agent)

# 시작 노드 설정
graph.set_entry_point("extractor")

# 노드 연결
graph.add_edge("extractor", "rag_search")
graph.add_edge("rag_search", "answer_generator")
graph.add_edge("answer_generator", "result_formatter")
graph.add_edge("result_formatter", END)  # END: 그래프 종료

# 그래프 컴파일
app = graph.compile()

# ================================================================
# 8. 실행 예시 (Jupyter Notebook)
# ================================================================
# 1) 시각화 UI 표시
visualizer.show()

# 2) RAG 파이프라인 실행
state = {
    "query": "한강에 버스가 있는 내용",  # 사용자 질의
    "keywords": "",            # 초기 키워드
    "search_results": [],      # 검색 결과
    "context": "",             # 문서 내용
    "result": ""               # 최종 답변
}

# 3) invoke()로 그래프 실행
result = app.invoke(state)

# 4) 최종 결과 출력
print("\n" + "="*50)
print("📊 RAG 처리 완료! 최종 결과:")
print("="*50)
print(result["result"])

# (visualizer는 에이전트 실행 도중 실시간으로 업데이트됩니다.)

    'data': [{'marker': {'color': [gray, gray, gray, gray],
     …


📊 RAG 처리 완료! 최종 결과:

🔍 검색 결과:

--- 1순위 (100.0%) ---
📄 파일명: 속도미달.docx
📁 위치: fileList/속도미달.docx
📝 요약: 서울시는 한강버스의 해상 시운전 결과를 알면서도 평균속력과 최대속력을 과장해 공개하며 시민들을 기만한 것으로 드러났다. 실제로 시운전에서 평균 최고속도는 15노트 미만으로, 계획된 17노트보다 낮게 나타났다. 서울시는 정식 운행 전 속도 조정에 대한 명확한 설명을 내놓지 않아 비판을 받았고, 이러한 문제점에도 불구하고 hurriedly 공식 발표를 진행하며...
🏷️ 유형: .docx
🕒 등록일: 2025-10-21

--- 2순위 (61.2%) ---
📄 파일명: 파렴치한.docx
📁 위치: fileList/파렴치한.docx
📝 요약: 지난달 20일, 유튜브 채널 '한문철 TV'에 제보된 사연에서, 주차 자리를 맡고 있던 아줌마와의 충돌로 인해 차주 A씨는 특수폭행으로 고소당했다. A씨는 아주머니가 자신의 차와 약간 접촉한 후에도 이를 무시하고 주차를 완료했으나, 5분 뒤 아주머니의 남편으로부터 고의적 폭행 주장에 대한 전화를 받고 고소를 당했다. A씨는 주차 시 주변 상황 확인 부족을 ...
🏷️ 유형: .docx
🕒 등록일: 2025-10-21

--- 3순위 (50.5%) ---
📄 파일명: IT기사.docx
📁 위치: fileList/IT기사.docx
📝 요약: 홍민택 카카오 최고제품책임자(CPO)가 사내 공지를 통해 카카오톡 개편으로 인한 사용자 불만에 사과하며, 기존 서비스 유지와 함께 친구탭 피드 노출 방식 변경 및 인스타그램식 콘텐츠 분리 배치 계획을 밝혔다. 트래픽 감소는 없다고 전망하면서도 사용자 불편 최소화에 초점을 맞추겠다고 강조했다. 개편 배경은 카카오의 성장 정체 상황에서의 혁신 필요성과 기존 메...
🏷️ 유형: .docx
🕒 등록일: 2025-10-21

--- 4순위 (45.1%) ---
📄 파일명: 2025년 신문 구독 지원 신청 전 필독사항.pdf

# 진행률, 진행단계 삭제

In [54]:
# ================================================================
# 노트북용 RAG + 3D 시각화 통합 코드
# - 질문(Query)을 중앙(0,0,0)에 고정
# - 유사도(관련성)가 높을수록 중앙에 가깝게 배치
# - 에이전트 시각화 및 프로세스 정보 제거
# ================================================================

from typing import TypedDict, List, Dict, Any
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
import pymysql

import plotly.graph_objects as go
import numpy as np
import ipywidgets as widgets
from IPython.display import display

# ================================================================
# 1. 상태 정의
# ================================================================
class AgentState(TypedDict):
    query: str
    keywords: str
    search_results: List[Dict[str, Any]]
    context: str
    result: str

# ================================================================
# 2. DB 설정
# ================================================================
DB_CONFIG = {
    "host": "localhost",
    "user": "admin",
    "password": "1qazZAQ!",
    "db": "final",
    "charset": "utf8mb4",
}

# ================================================================
# 3. Chroma/LLM 설정
# ================================================================
CHROMA_PATH = "./rag_chroma/documents/title_summary/"
EMBEDDINGS = OllamaEmbeddings(model="exaone3.5:2.4b")
LLM = Ollama(model="exaone3.5:2.4b")
CHAT_LLM = ChatOllama(model="exaone3.5:2.4b", temperature=0.1)

# ================================================================
# 4. 3D 시각화 클래스 (에이전트 제거)
# ================================================================
class RAGNotebookVisualizer:
    def __init__(self):
        # 질문(쿼리)을 중앙에 고정
        self.query_position = (0.0, 0.0, 0.0)

        # 3D 그래프 위젯 생성
        self.fig3d = go.FigureWidget()
        self._init_scene()

        # 관련성 막대 그래프
        self.bar_fig = go.FigureWidget(
            data=[go.Bar(x=[], y=[], text=[], textposition="auto")],
            layout=go.Layout(title="검색 결과 관련성 (%)", height=260, margin=dict(t=40)),
        )

        # UI 레이아웃 (로그 창 제거)
        left_panel = widgets.VBox([self.fig3d], layout=widgets.Layout(width="60%"))
        right_panel = widgets.VBox([self.bar_fig], layout=widgets.Layout(width="40%"))
        self.container = widgets.HBox([left_panel, right_panel])

    def _init_scene(self):
        """3D 장면 초기화"""
        # 1. 쿼리 노드 (중앙, 크기 작게)
        self.query_trace_idx = len(self.fig3d.data)
        self.fig3d.add_trace(go.Scatter3d(
            x=[0], y=[0], z=[0],
            mode="markers+text",
            marker=dict(size=12, color="orange", symbol="diamond"),
            text=["❓ Query"], textposition="top center", name="사용자 질문"
        ))

        # 2. 쿼리-결과 연결선 (빨간 점선)
        self.query_edge_trace_idx = len(self.fig3d.data)
        self.fig3d.add_trace(go.Scatter3d(
            x=[], y=[], z=[],
            mode="lines",
            line=dict(color="rgba(255,0,0,0.5)", width=2, dash="dot"),
            showlegend=False, hoverinfo="skip"
        ))

        # 3. 검색 결과 노드 (초기 비어있음)
        self.search_trace_idx = len(self.fig3d.data)
        self.fig3d.add_trace(go.Scatter3d(
            x=[], y=[], z=[],
            mode="markers+text",
            marker=dict(
                size=[], color=[], colorscale="Viridis",
                cmin=0, cmax=100, opacity=0.9, showscale=True,
                colorbar=dict(title="관련성(%)")
            ),
            text=[], textposition="top center", name="검색 결과"
        ))

        # 4. 레이아웃 설정 (간소화)
        self.fig3d.update_layout(
            title="🔍 질문 기반 문서 배치 시각화",
            scene=dict(
                xaxis=dict(visible=False),
                yaxis=dict(visible=False),
                zaxis=dict(title="거리"),
                camera=dict(eye=dict(x=1.1, y=1.1, z=0.8)),
            ),
            height=600,
            margin=dict(l=0, r=0, t=40, b=0),
        )

    def show(self):
        display(self.container)

    def update_search_results(self, results: List[Dict[str, Any]]):
        """검색 결과를 3D 그래프와 막대 차트에 업데이트"""
        res_sorted = sorted(results, key=lambda x: x.get("relevance", 0), reverse=True)
        n = len(res_sorted)
        if n == 0: return

        bx, by, bz = self.query_position
        min_dist, max_dist = 0.2, 3.0

        xs, ys, zs = [], [], []
        edge_xs, edge_ys, edge_zs = [], [], []
        sizes, colors, texts = [], [], []

        # 구면 좌표계를 이용해 배치
        golden_angle = np.pi * (3 - np.sqrt(5))
        for i, r in enumerate(res_sorted):
            rel = r.get("relevance", 0)
            dist = min_dist + (1 - rel / 100) * (max_dist - min_dist)

            theta = i * golden_angle
            phi = np.arccos(1 - 2 * (i + 0.5) / n)

            x = bx + dist * np.sin(phi) * np.cos(theta)
            y = by + dist * np.sin(phi) * np.sin(theta)
            z = bz + dist * np.cos(phi)

            xs.append(x); ys.append(y); zs.append(z)
            edge_xs.extend([bx, x, None]); edge_ys.extend([by, y, None]); edge_zs.extend([bz, z, None])
            sizes.append(max(5, rel / 5))
            colors.append(rel)
            texts.append(f"{r.get('file_name','doc')}<br>{rel}%")

        # 3D 그래프 업데이트
        with self.fig3d.batch_update():
            self.fig3d.data[self.query_edge_trace_idx].x = edge_xs
            self.fig3d.data[self.query_edge_trace_idx].y = edge_ys
            self.fig3d.data[self.query_edge_trace_idx].z = edge_zs
            self.fig3d.data[self.search_trace_idx].x = xs
            self.fig3d.data[self.search_trace_idx].y = ys
            self.fig3d.data[self.search_trace_idx].z = zs
            self.fig3d.data[self.search_trace_idx].marker.size = sizes
            self.fig3d.data[self.search_trace_idx].marker.color = colors
            self.fig3d.data[self.search_trace_idx].text = texts

        # 막대 차트 업데이트
        with self.bar_fig.batch_update():
            self.bar_fig.data[0].x = [r.get("file_name", "") for r in res_sorted]
            self.bar_fig.data[0].y = [r.get("relevance", 0) for r in res_sorted]
            self.bar_fig.data[0].marker.color = [r.get("relevance", 0) for r in res_sorted]
            self.bar_fig.data[0].marker.colorscale = "Viridis"
            self.bar_fig.data[0].marker.showscale = True

# ================================================================
# 5. 시각화 객체 생성
# ================================================================
visualizer = RAGNotebookVisualizer()

# ================================================================
# 6. RAG 에이전트 함수 (에이전트 시각화 제거)
# ================================================================
def extractor_agent(state: AgentState):
    # 키워드 추출만 실행
    prompt = PromptTemplate.from_template(
        "질문에서 검색 키워드 추출 (쉼표 구분):\n{query}"
    )
    keywords = LLM.invoke(prompt.format(query=state["query"]))
    return {**state, "keywords": keywords.strip()}

def rag_search_agent(state: AgentState):
    try:
        # 1. ChromaDB 로드
        vectorstore = Chroma(persist_directory=CHROMA_PATH, embedding_function=EMBEDDINGS)
        
        # 2. 검색
        results = vectorstore.similarity_search_with_score(state["keywords"], k=10)
        if not results: return {**state, "search_results": [], "context": ""}

        # 3. 문서 중복 제거
        best_doc_info = {}
        for doc, score in results:
            doc_id = doc.metadata.get("id")
            if doc_id:
                content = doc.page_content.strip()
                if content:
                    if doc_id not in best_doc_info or score < best_doc_info[doc_id][0]:
                        best_doc_info[doc_id] = (score, content)

        if not best_doc_info: return {**state, "search_results": [], "context": ""}

        # 4. 유사도 정규화
        scores = [score for score, _ in best_doc_info.values()]
        min_score, max_score = min(scores), max(scores)

        # 5. MySQL에서 데이터 조회
        conn = pymysql.connect(**DB_CONFIG)
        cursor = conn.cursor(pymysql.cursors.DictCursor)
        search_results = []
        
        for doc_id, (score, content) in best_doc_info.items():
            cursor.execute("SELECT * FROM documents WHERE id = %s", (doc_id,))
            row = cursor.fetchone()
            if row:
                relevance = (1 - (score - min_score) / (max_score - min_score)) * 100 if max_score != min_score else 100
                search_results.append({
                    "relevance": round(relevance, 1),
                    "file_name": row["file_name"],
                    "file_location": row["file_location"],
                    "summary": row["summary"][:100] + "..." if row["summary"] else "",
                    "doc_type": row["doc_type"],
                    "content": content
                })
        
        conn.close()
        
        # 6. 정렬 및 결과 반환
        search_results.sort(key=lambda x: x["relevance"], reverse=True)
        context = "\n".join([r["content"] for r in search_results[:10]])
        
        # 7. 시각화 업데이트 (에이전트 프로세스 표시 없이)
        visualizer.update_search_results(search_results)
        
        return {**state, "search_results": search_results, "context": context}
    
    except Exception as e:
        print(f"Error: {e}")
        return {**state, "search_results": [], "context": ""}

def answer_generator_agent(state: AgentState):
    if not state["search_results"]:
        return {**state, "result": "관련 정보 없음"}
    
    # 답변 생성
    search_summary = "\n".join([
        f"- {r['file_name']} ({r['relevance']}%)" 
        for r in state["search_results"]
    ])
    
    prompt = ChatPromptTemplate.from_template(
        "다음 문서를 참고하여 답변 생성:\n{search_summary}\n\n질문: {query}"
    )
    chain = prompt | CHAT_LLM | StrOutputParser()
    result = chain.invoke({
        "search_summary": search_summary,
        "query": state["query"]
    })
    
    return {**state, "result": result.strip()}

def result_formatter_agent(state: AgentState):
    formatted_result = "🔍 검색 결과:\n"
    for i, r in enumerate(state["search_results"], 1):
        formatted_result += f"{i}. {r['file_name']} ({r['relevance']}%)\n"
    
    formatted_result += f"\n💬 답변:\n{state['result']}"
    return {**state, "result": formatted_result}

# ================================================================
# 7. LangGraph 구성 및 실행
# ================================================================
from langgraph.graph import StateGraph, END

graph = StateGraph(AgentState)
graph.add_node("extractor", extractor_agent)
graph.add_node("rag_search", rag_search_agent)
graph.add_node("answer_generator", answer_generator_agent)
graph.add_node("result_formatter", result_formatter_agent)

graph.set_entry_point("extractor")
graph.add_edge("extractor", "rag_search")
graph.add_edge("rag_search", "answer_generator")
graph.add_edge("answer_generator", "result_formatter")
graph.add_edge("result_formatter", END)

app = graph.compile()

# ================================================================
# 8. 실행 예시
# ================================================================
# 1) 시각화 UI 표시
visualizer.show()

# 2) RAG 파이프라인 실행
state = {"query": "한강에 버스가 있는 내용", "keywords": "", "search_results": [], "context": "", "result": ""}
result = app.invoke(state)

# 3) 최종 결과 출력
print("\n" + "="*50)
print("최종 결과:")
print("="*50)
print(result["result"])

    'data': [{'marker': {'color': 'orange', 'size': 12, 'symbol':…


최종 결과:
🔍 검색 결과:
1. 속도미달.docx (100.0%)
2. 파렴치한.docx (70.5%)
3. 세계기사.docx (55.0%)
4. 소름돋는정책원해.docx (43.7%)
5. IT기사.docx (42.8%)
6. 2025년 신문 구독 지원 신청 전 필독사항.pdf (28.2%)
7. 전유성별세.docx (8.1%)
8. 국민의군대.docx (7.4%)
9. 사회기사정보.docx (2.4%)
10. 정치기사뉴스.docx (0.0%)

💬 답변:
제시된 문서들의 내용과 관련하여 "한강에 버스가 있는 내용"에 대한 직접적인 정보를 제공하기는 어렵습니다. 주어진 문서들은 다양한 주제와 점수를 가진 뉴스나 기사들에 대한 요약이나 평가를 나타내고 있지만, 특정 주제인 "한강에 버스가 있는 내용"에 대한 구체적인 내용이나 관련 문서는 포함되어 있지 않습니다.

만약 "한강에 버스가 있는 내용"에 대한 정보를 찾고 계신다면, 다음과 같은 접근 방법을 고려해 볼 수 있습니다:

1. **최신 뉴스 검색**: 인터넷 뉴스 사이트나 관련 앱을 통해 최근 한강에서 버스 운행 관련 뉴스나 기사를 검색해 보세요.
2. **지역 커뮤니티 또는 포럼**: 지역 커뮤니티나 온라인 포럼에서 한강 버스 운행에 대한 토론이나 정보를 찾아볼 수 있습니다.
3. **공식 발표 확인**: 서울시나 한강 관리 기관의 공식 웹사이트를 통해 공식적인 발표나 공지사항을 확인해 보세요.

이러한 방법들을 통해 더 구체적이고 관련성 있는 정보를 얻을 수 있을 것입니다. 만약 특정 문서나 주제에 대한 추가적인 정보가 필요하다면, 좀 더 구체적인 맥락이나 세부 사항을 제공해 주시면 도움을 드리겠습니다.


# 질문으로 부터 가까운 3D

In [32]:
# ================================================================
# 노트북용 RAG + 3D 시각화 통합 코드
# ================================================================
# 이 코드는 RAG(Retrieval-Augmented Generation) 시스템을 Jupyter Notebook 환경에서 
# 실행하고, 처리 과정을 3D 시각화하는 통합 솔루션입니다.
# - 사용자가 입력한 질의에 대해 관련 문서를 검색하고, AI가 답변을 생성하는 과정을
#   실시간으로 3D 그래프와 차트로 모니터링할 수 있습니다.
# - 시스템은 4개의 에이전트로 구성되어 있으며, LangGraph로 연결되어 순차적으로 실행됩니다.
# ================================================================

from typing import TypedDict, List, Dict, Any
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate  # 프롬프트 템플릿 생성
from langchain_core.output_parsers import StrOutputParser  # LLM 출력 파싱
from langchain_community.llms import Ollama  # Ollama LLM
from langchain_community.chat_models import ChatOllama  # Ollama 챗 모델
from langchain_community.vectorstores import Chroma  # 벡터 데이터베이스
from langchain_community.embeddings import OllamaEmbeddings  # Ollama 임베딩
from langgraph.graph import StateGraph, END  # LangGraph 컴포넌트
import pymysql  # MySQL 데이터베이스 연동

# 시각화 관련
import plotly.graph_objects as go  # 3D 그래프 생성
import plotly.express as px  # 간편한 시각화
import numpy as np  # 수치 연산
import ipywidgets as widgets  # 노트북 위젯
from IPython.display import display  # 노트북 출력
import datetime  # 시간 처리

# ================================================================
# 1. 상태 정의 (AgentState)
# ================================================================
# - 에이전트 간에 전달되는 상태를 정의하는 자료 구조
# - query: 사용자 입력
# - keywords: 추출된 검색 키워드
# - search_results: 검색된 문서 목록 (파일명, 위치, 요약, 관련성 등)
# - context: 검색된 문서들의 결합 내용
# - result: 최종 생성된 답변
# ================================================================
class AgentState(TypedDict):
    query: str
    keywords: str
    search_results: List[Dict[str, Any]]
    context: str
    result: str

# ================================================================
# 2. 데이터베이스 접속 정보
# ================================================================
# - MySQL 데이터베이스 연결을 위한 설정
# - host: localhost
# - user: admin
# - password: 1qazZAQ!
# - db: final
# - charset: utf8mb4 (한글 지원)
# ================================================================
DB_CONFIG = {
    'host': 'localhost',
    'user': 'admin',
    'password': '1qazZAQ!',
    'db': 'final',
    'charset': 'utf8mb4'
}

# ================================================================
# 3. ChromaDB 및 LLM 설정
# ================================================================
# - ChromaDB: 벡터 데이터베이스로 문서 임베딩 저장
# - OllamaEmbeddings: 문서 임베딩용 Ollama 모델 (exaone3.5:2.4b)
# - Ollama: 텍스트 생성용 Ollama 모델 (exaone3.5:2.4b)
# - ChatOllama: 챗봇용 Ollama 모델 (exaone3.5:2.4b, temperature=0.1)
# ================================================================
CHROMA_PATH = "./rag_chroma/documents/title_summary/"  # ChromaDB 저장 경로
EMBEDDINGS = OllamaEmbeddings(model="exaone3.5:2.4b")  # 임베딩 모델
LLM = Ollama(model="exaone3.5:2.4b")  # LLM
CHAT_LLM = ChatOllama(model="exaone3.5:2.4b", temperature=0.1)  # 챗 모델

# ================================================================
# 4. 노트북용 3D 시각화 클래스 (RAGNotebookVisualizer)
# ================================================================
# - RAG 처리 과정을 3D로 시각화하는 클래스
# - 에이전트 상태, 검색 결과, 진행률을 실시간으로 보여줌
# ================================================================
class RAGNotebookVisualizer:
    def __init__(self):
        # 에이전트의 3D 공간 위치 (x, y, z)
        self.agent_positions = {
            'extractor': (0.0, 0.0, 0.0),       # 키워드 추출
            'rag_search': (2.0, 0.0, 1.0),      # 문서 검색
            'answer_generator': (4.0, 0.0, 2.0), # 답변 생성
            'result_formatter': (6.0, 0.0, 1.0)  # 결과 형식화
        }
        self.agent_names = list(self.agent_positions.keys())
        # 에이전트 초기 상태 (대기중)
        self.agent_status = {name: 'waiting' for name in self.agent_names}
        
        # 쿼리 노드 위치 저장 (검색 결과 배치의 중심점)
        self.query_position = None # (x, y, z)

        # 3D FigureWidget 생성
        self.fig3d = go.FigureWidget()
        self._init_3d_scene()  # 3D 장면 초기화

        # 관련성 막대 그래프 (검색 결과)
        self.bar_fig = go.FigureWidget(
            data=[go.Bar(x=[], y=[], text=[], textposition='auto')],
            layout=go.Layout(
                title='검색 결과 관련성 (%)',
                height=260,
                margin=dict(t=40)
            )
        )

        # 진행률 바 (에이전트 진행 상황)
        self.progress = widgets.FloatProgress(
            value=0.0,
            min=0.0,
            max=100.0,
            description='진행률:',
            bar_style='info'
        )

        # 로그 텍스트 영역 (처리 기록)
        self.log = widgets.Textarea(
            value='',
            placeholder='로그가 여기에 표시됩니다.',
            layout=widgets.Layout(width='100%', height='600px')
        )

        # 전체 UI 레이아웃 (3D + 차트 + 로그)
        left_panel = widgets.VBox([self.fig3d], layout=widgets.Layout(width='60%'))
        right_panel = widgets.VBox([self.bar_fig, self.progress, self.log], layout=widgets.Layout(width='40%'))
        self.container = widgets.HBox([left_panel, right_panel])

    def _init_3d_scene(self):
        """3D 장면을 초기화하고 에이전트 노드/연결선을 추가"""
        # 에이전트 위치 (x, y, z)
        xs = [pos[0] for pos in self.agent_positions.values()]
        ys = [pos[1] for pos in self.agent_positions.values()]
        zs = [pos[2] for pos in self.agent_positions.values()]

        # 에이전트 노드 (마커 + 텍스트)
        node_trace = go.Scatter3d(
            x=xs,
            y=ys,
            z=zs,
            mode='markers+text',  # 마커와 텍스트 함께 표시
            marker=dict(
                size=14,            # 마커 크기
                color=['gray'] * len(self.agent_names),  # 초기 색상 (회색)
                opacity=0.9,         # 투명도
                line=dict(width=2, color='DarkSlateGrey')  # 마커 테두리
            ),
            text=self.agent_names,   # 노드 텍스트
            textposition="top center",  # 텍스트 위치
            textfont=dict(size=12, color='black'),
            name="에이전트"
        )
        self.fig3d.add_trace(node_trace)

        # 에이전트 간 연결선 (에지)
        agent_connection_trace_count = 0
        for i in range(len(self.agent_names) - 1):
            a = self.agent_positions[self.agent_names[i]]
            b = self.agent_positions[self.agent_names[i+1]]
            edge = go.Scatter3d(
                x=[a[0], b[0]],
                y=[a[1], b[1]],
                z=[a[2], b[2]],
                mode='lines',
                line=dict(color='lightgray', width=4),
                showlegend=False
            )
            self.fig3d.add_trace(edge)
            agent_connection_trace_count += 1

        # 🆕 쿼리 노드 추가 (placeholder)
        self.query_trace_idx = len(self.fig3d.data) # 현재 추가될 trace의 인덱스
        query_trace = go.Scatter3d(
            x=[], y=[], z=[],
            mode='markers+text',
            marker=dict(
                size=20,
                color='orange',
                opacity=1.0,
                symbol='diamond',
                line=dict(width=3, color='darkorange')
            ),
            text=[],
            textposition='top center',
            textfont=dict(size=14, color='darkorange', family='Arial Black'),
            name='사용자 쿼리',
            hovertemplate='<b>쿼리</b>: %{text}<extra></extra>'
        )
        self.fig3d.add_trace(query_trace)

        # 🆕 쿼리-검색결과 연결선 placeholder (초록색 점선)
        self.query_edge_trace_idx = len(self.fig3d.data)
        query_edge_trace = go.Scatter3d(
            x=[], y=[], z=[],
            mode='lines',
            line=dict(color='rgba(50, 205, 50, 0.5)', width=2, dash='dot'),  # 밝은 초록색
            showlegend=False,
            hoverinfo='skip'
        )
        self.fig3d.add_trace(query_edge_trace)

        # 검색 결과 placeholder (업데이트 예정)
        self.search_trace_idx = len(self.fig3d.data) # 검색 결과 trace의 인덱스
        search_trace = go.Scatter3d(
            x=[], y=[], z=[],
            mode='markers+text',
            marker=dict(
                size=[], 
                color=[], 
                colorscale='Viridis',  # 색상 척도 (관련성 높을수록 밝은 노란색)
                cmin=0, 
                cmax=100, 
                opacity=0.8, 
                showscale=True,       # 색상 바 표시
                colorbar=dict(title='관련성 (%)')
            ),
            text=[], 
            textposition='top center',
            name='검색 결과',
            hovertemplate='<b>%{text}</b><extra></extra>'
        )
        self.fig3d.add_trace(search_trace)

        # 3D 레이아웃 설정
        self.fig3d.update_layout(
            title='🤖 RAG 처리 과정 3D 시각화 (Notebook)',
            scene=dict(
               	xaxis=dict(title='⏩ 진행 단계'),      # 시간의 흐름
                yaxis=dict(title='🗂️ 배치 공간'),      # 단순 시각적 분산 공간
                zaxis=dict(title='⭐ 중요도 (관련성%)'), # 높이가 관련성 나타냄
                camera=dict(eye=dict(x=1.5, y=1.5, z=1.5)) # 초기 카메라 각도
            ),
            height=600,
            margin=dict(l=0, r=0, t=40, b=0),  # 마진
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)  # 범례
        )

    def show(self):
        """시각화 UI를 노트북에 표시"""
        display(self.container)

    def _append_log(self, text: str):
        """로그 메시지를 추가하고 표시"""
        # 현재 시간 포맷: HH:MM:SS
        timestamp = datetime.datetime.now().strftime("%H:%M:%S")
        self.log.value += f"[{timestamp}] {text}\n"

    def update_agent_status(self, agent_name: str, status: str, data: List[Dict[str, Any]] = None, query: str = None):
        """
        에이전트 상태를 업데이트하고 시각화를 갱신
        
        Args:
            agent_name: 업데이트할 에이전트 이름
            status: 'processing', 'completed', 'error', 'waiting'
            data: 검색 결과 데이터 (rag_search 에이전트 전용)
            query: 사용자 쿼리 (extractor 에이전트 실행 시 전달)
        """
        # 🆕 쿼리 텍스트를 받아 쿼리 노드 위치 및 텍스트 업데이트
        if query:
            self.current_query = query
            # 쿼리 노드의 위치를 고정하거나, 파이프라인 시작점 근처에 배치
            # 예: extractor 에이전트 위치를 기준으로 약간 왼쪽으로 배치
            query_x = self.agent_positions['extractor'][0] - 2.0
            query_y = self.agent_positions['extractor'][1] # 같은 높이
            query_z = self.agent_positions['extractor'][2] # 같은 높이
            self.query_position = (query_x, query_y, query_z)
            self._update_query_node(query, self.query_position)
        
        # 1. 로그에 상태 추가
        status_text = status.capitalize()
        self._append_log(f"{agent_name} -> {status_text}")

        # 2. 에이전트 상태 저장
        if agent_name in self.agent_status:
            self.agent_status[agent_name] = status

        # 3. 색상 매핑: 상태에 따른 색상
        def get_status_color(s):
            if s == 'processing': return 'yellow'
            elif s == 'completed': return 'green'
            elif s == 'error': return 'red'
            else: return 'gray'  # waiting

        # 4. 3D 노드 색상 업데이트
        colors = [get_status_color(self.agent_status[name]) for name in self.agent_names]
        self.fig3d.data[0].marker.color = colors

        # 5. 진행률 업데이트 (완료된 에이전트 비율)
        completed_count = sum(1 for s in self.agent_status.values() if s == 'completed')
        progress_percent = (completed_count / len(self.agent_status)) * 100
        self.progress.value = progress_percent

        # 6. 검색 결과가 있는 경우 시각화 업데이트
        if data and agent_name == 'rag_search':
            self.update_search_results(data)

    def _update_query_node(self, query: str, position: tuple):
        """쿼리 노드를 3D 공간에 업데이트"""
        query_x, query_y, query_z = position
        # 쿼리 텍스트 (최대 30자로 제한)
        query_text = query if len(query) <= 30 else query[:27] + "..."
        
        # 쿼리 노드 trace 업데이트 (저장된 인덱스 사용)
        trace = self.fig3d.data[self.query_trace_idx]
        trace.x = [query_x]
        trace.y = [query_y]
        trace.z = [query_z]
        trace.text = [f"🔍 {query_text}"]

    def update_search_results(self, results: List[Dict[str, Any]]):
        """검색 결과를 3D 그래프와 막대 차트에 업데이트
        
        Args:
            results: 검색 결과 목록 (relevance, file_name, ...)
        """
        # 1. 관련성 순으로 내림차순 정렬
        results_sorted = sorted(results, key=lambda x: x.get('relevance', 0), reverse=True)
        
        # 2. 3D 검색 결과 배치 중심점 설정
        if not self.query_position:
            # query_position이 설정되지 않았다면 fallback으로 rag_search 위치 사용
            base_x, base_y, base_z = self.agent_positions['rag_search']
            print("Warning: Query position not set, using rag_search agent as center for results.")
        else:
            base_x, base_y, base_z = self.query_position # 쿼리 노드의 위치를 중심으로 사용

        n = len(results_sorted)
        xs, ys, zs, sizes, colors, texts = [], [], [], [], [], []
        radius = 1.5  # 쿼리 노드 중심 주변의 원형 배치 반지름

        # 🆕 쿼리-검색결과 연결선 좌표 저장
        edge_xs, edge_ys, edge_zs = [], [], []

        # 3. 검색 결과를 원형으로 배치 (query position 기준)
        for i, res in enumerate(results_sorted):
            angle = (i * 2 * np.pi) / n  # 각도 계산
            
            # 3D 좌표: query position을 중심으로 원형 배치
            x = base_x + radius * np.cos(angle)
            y = base_y + radius * np.sin(angle)
            # Z축(중요도): 쿼리 노드의 Z 위치 + 관련성에 따른 높이
            z = base_z + (res.get('relevance', 0) / 100) * 2  # 관련성에 따른 높이

            xs.append(x)
            ys.append(y)
            zs.append(z)
            sizes.append(max(6, res.get('relevance', 0) / 4))  # 관련성에 따른 마커 크기
            colors.append(res.get('relevance', 0))  # 색상 (관련성 값)
            # 텍스트: 파일명 + 관련성
            text = f"{res.get('file_name', '파일')}<br>{res.get('relevance', 0)}%"
            texts.append(text)
            
            # 🆕 쿼리 노드에서 각 검색 결과 노드로 연결선 추가
            edge_xs.extend([base_x, x, None])  # None으로 각 선을 분리
            edge_ys.extend([base_y, y, None])
            edge_zs.extend([base_z, z, None])

        # 🆕 쿼리-검색결과 연결선 업데이트 (저장된 인덱스 사용)
        trace_edge = self.fig3d.data[self.query_edge_trace_idx]
        trace_edge.x = edge_xs
        trace_edge.y = edge_ys
        trace_edge.z = edge_zs

        # 4. 3D 검색 결과 trace 업데이트 (저장된 인덱스 사용)
        trace_search = self.fig3d.data[self.search_trace_idx]
        trace_search.x = xs
        trace_search.y = ys
        trace_search.z = zs
        trace_search.marker.size = sizes
        trace_search.marker.color = colors
        trace_search.text = texts
        
        # 5. 막대 차트 업데이트 (관련성) - 이 부분은 변경 없음
        file_names = [res.get('file_name', '') for res in results_sorted]
        relevances = [res.get('relevance', 0) for res in results_sorted]

        with self.bar_fig.batch_update():
            self.bar_fig.data[0].x = file_names
            self.bar_fig.data[0].y = relevances
            self.bar_fig.data[0].text = [f"{r}%" for r in relevances]
            self.bar_fig.data[0].marker.color = relevances
            self.bar_fig.data[0].marker.colorscale = 'Viridis'
            self.bar_fig.data[0].marker.cmin = 0
            self.bar_fig.data[0].marker.cmax = 100
            self.bar_fig.data[0].marker.showscale = True

        # 6. 막대 차트 레이아웃 업데이트 (너비 조절)
        self.bar_fig.update_layout(
            xaxis_tickangle=-45,
            margin=dict(b=100)
        )


# ================================================================
# 5. 시각화 객체 생성
# ================================================================
# - 위에서 정의한 RAGNotebookVisualizer 인스턴스 생성
# - 노트북에서 show() 메소드를 호출해 UI 표시
# ================================================================
visualizer = RAGNotebookVisualizer()

# ================================================================
# 6. RAG 에이전트 함수
# ================================================================
# - 4개의 에이전트: 키워드 추출, RAG 검색, 답변 생성, 결과 형식화
# - 각 에이전트는 visualizer.update_agent_status()로 상태를 업데이트
# ================================================================

# 6.1. 키워드 추출 에이전트
def extractor_agent(state: AgentState):
    # 상태: processing (🆕 쿼리 전달)
    # extractor 에이전트가 실행될 때, 현재 쿼리를 visualizer에 전달하여 쿼리 노드 위치를 설정합니다.
    visualizer.update_agent_status('extractor', 'processing', query=state["query"])
    
    # 프롬프트 템플릿: 사용자 질의에서 키워드 추출
    keyword_prompt = PromptTemplate.from_template(
        """사용자의 질문에서 띄어쓰기 확인하고 찾고자 하는 키워드를 쉼표로 구분하여 출력하세요.
        벡터스토어 검색을 위한 최적의 키워드를 추출해주세요.
        \n질문: {query}"""
    )
    # 프롬프트 포맷팅
    formatted_prompt = keyword_prompt.format(query=state["query"])
    # LLM 호출
    keywords = LLM.invoke(formatted_prompt)
    
    # 상태: completed
    visualizer.update_agent_status('extractor', 'completed')
    # state 업데이트 (keywords 추가)
    return {**state, "keywords": keywords.strip()}

# 6.2. RAG 검색 에이전트
def rag_search_agent(state: AgentState):
    # 상태: processing
    visualizer.update_agent_status('rag_search', 'processing')
    
    try:
        # 1. ChromaDB 로드
        vectorstore = Chroma(persist_directory=CHROMA_PATH, embedding_function=EMBEDDINGS)
        # 2. 벡터스토어 검색 (k=10: 상위 10개)
        results = vectorstore.similarity_search_with_score(state["keywords"], k=10)

        if not results:
            # 검색 결과 없음
            visualizer.update_agent_status('rag_search', 'completed', [])
            return {**state, "search_results": [], "context": ""}

        # 3. 문서 중복 제거 (최고 유사도 유지)
        best_doc_info = {}
        for doc, score in results:
            doc_id = doc.metadata.get("id")  # 문서 ID
            if doc_id:
                content = doc.page_content.strip()
                if not content:  # 내용이 빈 경우 스킵
                    continue
                # 동일 문서 ID 중 가장 높은 유사도를 유지
                if doc_id not in best_doc_info or score < best_doc_info[doc_id][0]:
                    best_doc_info[doc_id] = (score, content)

        if not best_doc_info:
            visualizer.update_agent_status('rag_search', 'completed', [])
            return {**state, "search_results": [], "context": ""}

        # 4. 유사도 정규화 (0-100%로 변환)
        scores = [score for score, _ in best_doc_info.values()]
        min_score = min(scores)
        max_score = max(scores)

        # 5. MySQL에서 파일 메타데이터 조회
        conn = None
        search_results = []
        try:
            conn = pymysql.connect(**DB_CONFIG)
            cursor = conn.cursor(pymysql.cursors.DictCursor)

            for doc_id, (score, content) in best_doc_info.items():
                # MySQL: documents 테이블에서 문서 ID로 조회
                cursor.execute("SELECT * FROM documents WHERE id = %s", (doc_id,))
                row = cursor.fetchone()
                if not row:
                    continue  # 데이터 없을 경우 스킵

                # 6. 관련성 % 계산
                if min_score == max_score:
                    relevance = 100.0  # 모든 문서 동일 유사도
                else:
                    # min-max normalization: 0-100%로 변환
                    relevance = round((1 - (score - min_score) / (max_score - min_score)) * 100, 1)

                # 7. 검색 결과 목록에 추가
                search_results.append({
                    "relevance": relevance,
                    "file_name": row["file_name"],
                    "file_location": row["file_location"],
                    "summary": (row["summary"][:200] + "...") if row["summary"] else "요약 없음",
                    "title": row["title"],
                    "created_at": row["created_at"].strftime("%Y-%m-%d") if row["created_at"] else "N/A",
                    "doc_type": row["doc_type"],
                    "content": content
                })

            # 8. 관련성 순으로 정렬 (내림차순) 및 상위 10개
            search_results.sort(key=lambda x: x["relevance"], reverse=True)
            search_results = search_results[:10]
            # 9. 문서 내용 결합 (context)
            context = "\n\n".join([res['content'] for res in search_results])

        except Exception as e:
            # MySQL 오류 처리
            print(f"❌ MySQL 오류: {e}")
            visualizer.update_agent_status('rag_search', 'error')
            return {**state, "search_results": [], "context": ""}
        finally:
            if conn:
                conn.close()

        # 상태: completed + 검색 결과 데이터 전달
        visualizer.update_agent_status('rag_search', 'completed', search_results)
        return {**state, "search_results": search_results, "context": context}

    except Exception as e:
        # 일반 오류 처리
        print(f"❌ RAG 검색 오류: {e}")
        visualizer.update_agent_status('rag_search', 'error')
        return {**state, "search_results": [], "context": ""}

# 6.3. 답변 생성 에이전트
def answer_generator_agent(state: AgentState):
    # 상태: processing
    visualizer.update_agent_status('answer_generator', 'processing')
    
    # 검색 결과가 없는 경우
    if not state["search_results"]:
        visualizer.update_agent_status('answer_generator', 'completed')
        return {**state, "result": "관련 정보를 찾을 수 없습니다."}

    try:
        # 1. 프롬프트 템플릿
        prompt = ChatPromptTemplate.from_template(
            """다음 문서 내용들을 바탕으로 사용자 질문에 답변해 주세요.
            - 문서에 직접 언급된 내용만을 바탕으로 답변해야 합니다.
            - 정보가 없는 경우 "정보가 없습니다"라고 명확히 답변해 주세요.
            - 항상 공손하고 전문적인 어조를 유지해 주세요.

            ### 검색 결과 요약:
            {search_summary}

            ### 문서 내용:
            {context}

            ### 사용자 질문:
            {query}

            ### 답변:
            찾은 문서중에 몇번째 정보가 가장 관련이 높은지를 상세히 답변해 주세요.
            (몇번째, 무슨파일인지, 어디에 있는지, 요약은 무엇인지 등)
            """
        )

        # 2. 검색 결과 요약 생성
        search_summary = "\n".join([
            f"{i+1}순위 ({res['relevance']}%): {res['file_name']} ({res['doc_type']}) - {res['summary']}"
            for i, res in enumerate(state["search_results"])
        ])

        # 3. LLM 체인: 프롬프트 + LLM + 출력 파서
        chain = prompt | CHAT_LLM | StrOutputParser()
        result = chain.invoke({
            "search_summary": search_summary,
            "context": state["context"],
            "query": state["query"]
        })

        # 상태: completed
        visualizer.update_agent_status('answer_generator', 'completed')
        return {**state, "result": result.strip()}

    except Exception as e:
        print(f"❌ 답변 생성 오류: {e}")
        visualizer.update_agent_status('answer_generator', 'error')
        return {**state, "result": "답변을 생성하는 중에 오류가 발생했습니다."}

# 6.4. 결과 형식화 에이전트
def result_formatter_agent(state: AgentState):
    # 상태: processing
    visualizer.update_agent_status('result_formatter', 'processing')
    
    # 검색 결과가 없는 경우
    if not state["search_results"]:
        visualizer.update_agent_status('result_formatter', 'completed')
        return state

    try:
        # 1. 검색 결과 형식화 (Markdown 스타일)
        formatted_result = "\n🔍 검색 결과:\n"
        for i, res in enumerate(state["search_results"], 1):
            formatted_result += f"\n--- {i}순위 ({res['relevance']}%) ---\n"
            formatted_result += f"📄 파일명: {res['file_name']}\n"
            formatted_result += f"📁 위치: {res['file_location']}\n"
            formatted_result += f"📝 요약: {res['summary']}\n"
            formatted_result += f"🏷️ 유형: {res['doc_type']}\n"
            formatted_result += f"🕒 등록일: {res['created_at']}\n"

        formatted_result += f"\n💬 AI 답변:\n{state['result']}"

        # 상태: completed
        visualizer.update_agent_status('result_formatter', 'completed')
        return {**state, "result": formatted_result}

    except Exception as e:
        print(f"❌ 결과 형식화 오류: {e}")
        visualizer.update_agent_status('result_formatter', 'error')
        return state

# ================================================================
# 7. LangGraph 구성
# ================================================================
# - 에이전트들을 노드로 연결한 그래프 생성
# - 실행 순서: extractor → rag_search → answer_generator → result_formatter
# ================================================================
graph = StateGraph(AgentState)  # AgentState를 상태로 사용하는 그래프
graph.add_node("extractor", extractor_agent)  # 노드 추가
graph.add_node("rag_search", rag_search_agent)
graph.add_node("answer_generator", answer_generator_agent)
graph.add_node("result_formatter", result_formatter_agent)

# 시작 노드 설정
graph.set_entry_point("extractor")

# 노드 연결
graph.add_edge("extractor", "rag_search")
graph.add_edge("rag_search", "answer_generator")
graph.add_edge("answer_generator", "result_formatter")
graph.add_edge("result_formatter", END)  # END: 그래프 종료

# 그래프 컴파일
app = graph.compile()

# ================================================================
# 8. 실행 예시 (Jupyter Notebook)
# ================================================================
# 1) 시각화 UI 표시
visualizer.show()

# 2) RAG 파이프라인 실행
state = {
    "query": "한강에 버스가 있는 내용",  # 사용자 질의
    "keywords": "",            # 초기 키워드
    "search_results": [],      # 검색 결과
    "context": "",             # 문서 내용
    "result": ""               # 최종 답변
}

# 3) invoke()로 그래프 실행
result = app.invoke(state)

# 4) 최종 결과 출력
print("\n" + "="*50)
print("📊 RAG 처리 완료! 최종 결과:")
print("="*50)
print(result["result"])

# (visualizer는 에이전트 실행 도중 실시간으로 업데이트됩니다.)

    'data': [{'marker': {'color': [gray, gray, gray, gray],
     …


📊 RAG 처리 완료! 최종 결과:

🔍 검색 결과:

--- 1순위 (100.0%) ---
📄 파일명: 속도미달.docx
📁 위치: fileList/속도미달.docx
📝 요약: 서울시는 한강버스의 해상 시운전 결과를 알면서도 평균속력과 최대속력을 과장해 공개하며 시민들을 기만한 것으로 드러났다. 실제로 시운전에서 평균 최고속도는 15노트 미만으로, 계획된 17노트보다 낮게 나타났다. 서울시는 정식 운행 전 속도 조정에 대한 명확한 설명을 내놓지 않아 비판을 받았고, 이러한 문제점에도 불구하고 hurriedly 공식 발표를 진행하며...
🏷️ 유형: .docx
🕒 등록일: 2025-10-21

--- 2순위 (56.2%) ---
📄 파일명: IT기사.docx
📁 위치: fileList/IT기사.docx
📝 요약: 홍민택 카카오 최고제품책임자(CPO)가 사내 공지를 통해 카카오톡 개편으로 인한 사용자 불만에 사과하며, 기존 서비스 유지와 함께 친구탭 피드 노출 방식 변경 및 인스타그램식 콘텐츠 분리 배치 계획을 밝혔다. 트래픽 감소는 없다고 전망하면서도 사용자 불편 최소화에 초점을 맞추겠다고 강조했다. 개편 배경은 카카오의 성장 정체 상황에서의 혁신 필요성과 기존 메...
🏷️ 유형: .docx
🕒 등록일: 2025-10-21

--- 3순위 (53.3%) ---
📄 파일명: 파렴치한.docx
📁 위치: fileList/파렴치한.docx
📝 요약: 지난달 20일, 유튜브 채널 '한문철 TV'에 제보된 사연에서, 주차 자리를 맡고 있던 아줌마와의 충돌로 인해 차주 A씨는 특수폭행으로 고소당했다. A씨는 아주머니가 자신의 차와 약간 접촉한 후에도 이를 무시하고 주차를 완료했으나, 5분 뒤 아주머니의 남편으로부터 고의적 폭행 주장에 대한 전화를 받고 고소를 당했다. A씨는 주차 시 주변 상황 확인 부족을 ...
🏷️ 유형: .docx
🕒 등록일: 2025-10-21

--- 4순위 (35.5%) ---
📄 파일명: 세계기사.docx
📁 위치: fileList/세계기

# 질문 유사도 관련 3D

In [34]:
# ================================================================
# 노트북용 RAG + 3D 시각화 통합 코드
# ================================================================
# 이 코드는 RAG(Retrieval-Augmented Generation) 시스템을 Jupyter Notebook 환경에서 
# 실행하고, 처리 과정을 3D 시각화하는 통합 솔루션입니다.
# - 사용자가 입력한 질의에 대해 관련 문서를 검색하고, AI가 답변을 생성하는 과정을
#   실시간으로 3D 그래프와 차트로 모니터링할 수 있습니다.
# - 시스템은 4개의 에이전트로 구성되어 있으며, LangGraph로 연결되어 순차적으로 실행됩니다.
# ================================================================

from typing import TypedDict, List, Dict, Any
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate  # 프롬프트 템플릿 생성
from langchain_core.output_parsers import StrOutputParser  # LLM 출력 파싱
from langchain_community.llms import Ollama  # Ollama LLM
from langchain_community.chat_models import ChatOllama  # Ollama 챗 모델
from langchain_community.vectorstores import Chroma  # 벡터 데이터베이스
from langchain_community.embeddings import OllamaEmbeddings  # Ollama 임베딩
from langgraph.graph import StateGraph, END  # LangGraph 컴포넌트
import pymysql  # MySQL 데이터베이스 연동

# 시각화 관련
import plotly.graph_objects as go  # 3D 그래프 생성
import plotly.express as px  # 간편한 시각화
import numpy as np  # 수치 연산
import ipywidgets as widgets  # 노트북 위젯
from IPython.display import display  # 노트북 출력
import datetime  # 시간 처리

# ================================================================
# 1. 상태 정의 (AgentState)
# ================================================================
class AgentState(TypedDict):
    query: str
    keywords: str
    search_results: List[Dict[str, Any]]
    context: str
    result: str

# ================================================================
# 2. 데이터베이스 접속 정보
# ================================================================
DB_CONFIG = {
    'host': 'localhost',
    'user': 'admin',
    'password': '1qazZAQ!',
    'db': 'final',
    'charset': 'utf8mb4'
}

# ================================================================
# 3. ChromaDB 및 LLM 설정
# ================================================================
CHROMA_PATH = "./rag_chroma/documents/title_summary/"
EMBEDDINGS = OllamaEmbeddings(model="exaone3.5:2.4b")
LLM = Ollama(model="exaone3.5:2.4b")
CHAT_LLM = ChatOllama(model="exaone3.5:2.4b", temperature=0.1)

# ================================================================
# 4. 노트북용 3D 시각화 클래스 (RAGNotebookVisualizer)
# ================================================================
class RAGNotebookVisualizer:
    def __init__(self):
        self.agent_positions = {
            'extractor': (0.0, 0.0, 0.0),
            'rag_search': (2.0, 0.0, 1.0),
            'answer_generator': (4.0, 0.0, 2.0),
            'result_formatter': (6.0, 0.0, 1.0)
        }
        self.agent_names = list(self.agent_positions.keys())
        self.agent_status = {name: 'waiting' for name in self.agent_names}
        self.query_position = None
        self.current_query = ""

        self.fig3d = go.FigureWidget()
        self._init_3d_scene()

        self.bar_fig = go.FigureWidget(
            data=[go.Bar(x=[], y=[], text=[], textposition='auto')],
            layout=go.Layout(title='검색 결과 관련성 (%)', height=260, margin=dict(t=40))
        )

        self.progress = widgets.FloatProgress(value=0.0, min=0.0, max=100.0, description='진행률:', bar_style='info')
        self.log = widgets.Textarea(value='', placeholder='로그가 여기에 표시됩니다.', layout=widgets.Layout(width='100%', height='600px'))

        left_panel = widgets.VBox([self.fig3d], layout=widgets.Layout(width='60%'))
        right_panel = widgets.VBox([self.bar_fig, self.progress, self.log], layout=widgets.Layout(width='40%'))
        self.container = widgets.HBox([left_panel, right_panel])

    def _init_3d_scene(self):
        xs = [pos[0] for pos in self.agent_positions.values()]
        ys = [pos[1] for pos in self.agent_positions.values()]
        zs = [pos[2] for pos in self.agent_positions.values()]

        # 에이전트 노드
        node_trace = go.Scatter3d(
            x=xs, y=ys, z=zs, mode='markers+text',
            marker=dict(size=14, color=['gray']*len(self.agent_names), opacity=0.9, line=dict(width=2, color='DarkSlateGrey')),
            text=self.agent_names, textposition="top center", textfont=dict(size=12, color='black'), name="에이전트"
        )
        self.fig3d.add_trace(node_trace)

        # 에이전트 간 연결선
        for i in range(len(self.agent_names) - 1):
            a = self.agent_positions[self.agent_names[i]]
            b = self.agent_positions[self.agent_names[i+1]]
            edge = go.Scatter3d(
                x=[a[0], b[0]], y=[a[1], b[1]], z=[a[2], b[2]], mode='lines',
                line=dict(color='lightgray', width=4), showlegend=False
            )
            self.fig3d.add_trace(edge)

        # 쿼리 노드 placeholder
        self.query_trace_idx = len(self.fig3d.data)
        query_trace = go.Scatter3d(
            x=[], y=[], z=[], mode='markers+text',
            marker=dict(size=20, color='orange', opacity=1.0, symbol='diamond', line=dict(width=3, color='darkorange')),
            text=[], textposition='top center', textfont=dict(size=14, color='darkorange', family='Arial Black'),
            name='사용자 쿼리', hovertemplate='<b>쿼리</b>: %{text}<extra></extra>'
        )
        self.fig3d.add_trace(query_trace)

        # 쿼리-검색결과 연결선 (빨간색 점선)
        self.query_edge_trace_idx = len(self.fig3d.data)
        query_edge_trace = go.Scatter3d(
            x=[], y=[], z=[], mode='lines',
            line=dict(color='rgba(255, 0, 0, 0.6)', width=3, dash='dot'),
            showlegend=False, hoverinfo='skip'
        )
        self.fig3d.add_trace(query_edge_trace)

        # 검색 결과 placeholder
        self.search_trace_idx = len(self.fig3d.data)
        search_trace = go.Scatter3d(
            x=[], y=[], z=[], mode='markers+text',
            marker=dict(size=[], color=[], colorscale='Viridis', cmin=0, cmax=100, opacity=0.8, showscale=True, colorbar=dict(title='관련성 (%)')),
            text=[], textposition='top center', name='검색 결과', hovertemplate='<b>%{text}</b><extra></extra>'
        )
        self.fig3d.add_trace(search_trace)

        # 3D 레이아웃 설정
        self.fig3d.update_layout(
            title='🤖 RAG 처리 과정 3D 시각화 (Notebook)',
            scene=dict(
                xaxis=dict(title='', showticklabels=False),
                yaxis=dict(title='', showticklabels=False),
                zaxis=dict(title='⭐ 중요도 (관련성%)'),
                camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
            ),
            height=600, margin=dict(l=0, r=0, t=40, b=0),
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
        )

    def show(self):
        display(self.container)

    def _append_log(self, text: str):
        timestamp = datetime.datetime.now().strftime("%H:%M:%S")
        self.log.value += f"[{timestamp}] {text}\n"

    def update_agent_status(self, agent_name: str, status: str, data: List[Dict[str, Any]] = None, query: str = None):
        if query:
            self.current_query = query
            query_x = self.agent_positions['extractor'][0] - 2.0
            query_y = self.agent_positions['extractor'][1]
            query_z = self.agent_positions['extractor'][2]
            self.query_position = (query_x, query_y, query_z)
            self._update_query_node(query, self.query_position)

        status_text = status.capitalize()
        self._append_log(f"{agent_name} -> {status_text}")

        if agent_name in self.agent_status:
            self.agent_status[agent_name] = status

        def get_status_color(s):
            if s == 'processing': return 'yellow'
            elif s == 'completed': return 'green'
            elif s == 'error': return 'red'
            else: return 'gray'

        colors = [get_status_color(self.agent_status[name]) for name in self.agent_names]
        self.fig3d.data[0].marker.color = colors

        completed_count = sum(1 for s in self.agent_status.values() if s == 'completed')
        self.progress.value = (completed_count / len(self.agent_status)) * 100

        if data and agent_name == 'rag_search':
            self.update_search_results(data)

    def _update_query_node(self, query: str, position: tuple):
        query_x, query_y, query_z = position
        query_text = query if len(query) <= 30 else query[:27] + "..."
        trace = self.fig3d.data[self.query_trace_idx]
        trace.x = [query_x]
        trace.y = [query_y]
        trace.z = [query_z]
        trace.text = [f"🔍 {query_text}"]

    def update_search_results(self, results: List[Dict[str, Any]]):
        results_sorted = sorted(results, key=lambda x: x.get('relevance', 0), reverse=True)
        
        if not self.query_position:
            base_x, base_y, base_z = self.agent_positions['rag_search']
        else:
            base_x, base_y, base_z = self.query_position

        n = len(results_sorted)
        xs, ys, zs, sizes, colors, texts = [], [], [], [], [], []
        edge_xs, edge_ys, edge_zs = [], [], []

        max_radius = 2.0
        min_radius = 0.3

        for i, res in enumerate(results_sorted):
            relevance = res.get('relevance', 0)
            dynamic_radius = max_radius - (relevance / 100) * (max_radius - min_radius)
            angle = (i * 2 * np.pi) / n

            x = base_x + dynamic_radius * np.cos(angle)
            y = base_y + dynamic_radius * np.sin(angle)
            z = base_z + (relevance / 100) * 2

            xs.append(x)
            ys.append(y)
            zs.append(z)
            sizes.append(max(8, relevance / 3))
            colors.append(relevance)
            text = f"{res.get('file_name', '파일')}<br>{relevance}%"
            texts.append(text)

            edge_xs.extend([base_x, x, None])
            edge_ys.extend([base_y, y, None])
            edge_zs.extend([base_z, z, None])

        trace_edge = self.fig3d.data[self.query_edge_trace_idx]
        trace_edge.x = edge_xs
        trace_edge.y = edge_ys
        trace_edge.z = edge_zs

        trace_search = self.fig3d.data[self.search_trace_idx]
        trace_search.x = xs
        trace_search.y = ys
        trace_search.z = zs
        trace_search.marker.size = sizes
        trace_search.marker.color = colors
        trace_search.text = texts

        file_names = [res.get('file_name', '') for res in results_sorted]
        relevances = [res.get('relevance', 0) for res in results_sorted]

        with self.bar_fig.batch_update():
            self.bar_fig.data[0].x = file_names
            self.bar_fig.data[0].y = relevances
            self.bar_fig.data[0].text = [f"{r}%" for r in relevances]
            self.bar_fig.data[0].marker.color = relevances
            self.bar_fig.data[0].marker.colorscale = 'Viridis'
            self.bar_fig.data[0].marker.cmin = 0
            self.bar_fig.data[0].marker.cmax = 100
            self.bar_fig.data[0].marker.showscale = True

        self.bar_fig.update_layout(xaxis_tickangle=-45, margin=dict(b=100))

# ================================================================
# 5. 시각화 객체 생성
# ================================================================
visualizer = RAGNotebookVisualizer()

# ================================================================
# 6. RAG 에이전트 함수
# ================================================================

def extractor_agent(state: AgentState):
    visualizer.update_agent_status('extractor', 'processing', query=state["query"])
    
    keyword_prompt = PromptTemplate.from_template(
        """사용자의 질문에서 띄어쓰기 확인하고 찾고자 하는 키워드를 쉼표로 구분하여 출력하세요.
        벡터스토어 검색을 위한 최적의 키워드를 추출해주세요.
        \n질문: {query}"""
    )
    formatted_prompt = keyword_prompt.format(query=state["query"])
    keywords = LLM.invoke(formatted_prompt)
    
    visualizer.update_agent_status('extractor', 'completed')
    return {**state, "keywords": keywords.strip()}

def rag_search_agent(state: AgentState):
    visualizer.update_agent_status('rag_search', 'processing')
    
    try:
        vectorstore = Chroma(persist_directory=CHROMA_PATH, embedding_function=EMBEDDINGS)
        results = vectorstore.similarity_search_with_score(state["keywords"], k=10)

        if not results:
            visualizer.update_agent_status('rag_search', 'completed', [])
            return {**state, "search_results": [], "context": ""}

        best_doc_info = {}
        for doc, score in results:
            doc_id = doc.metadata.get("id")
            if doc_id:
                content = doc.page_content.strip()
                if not content:
                    continue
                if doc_id not in best_doc_info or score < best_doc_info[doc_id][0]:
                    best_doc_info[doc_id] = (score, content)

        if not best_doc_info:
            visualizer.update_agent_status('rag_search', 'completed', [])
            return {**state, "search_results": [], "context": ""}

        scores = [score for score, _ in best_doc_info.values()]
        min_score = min(scores)
        max_score = max(scores)

        conn = None
        search_results = []
        try:
            conn = pymysql.connect(**DB_CONFIG)
            cursor = conn.cursor(pymysql.cursors.DictCursor)

            for doc_id, (score, content) in best_doc_info.items():
                cursor.execute("SELECT * FROM documents WHERE id = %s", (doc_id,))
                row = cursor.fetchone()
                if not row:
                    continue

                if min_score == max_score:
                    relevance = 100.0
                else:
                    relevance = round((1 - (score - min_score) / (max_score - min_score)) * 100, 1)

                search_results.append({
                    "relevance": relevance,
                    "file_name": row["file_name"],
                    "file_location": row["file_location"],
                    "summary": (row["summary"][:200] + "...") if row["summary"] else "요약 없음",
                    "title": row["title"],
                    "created_at": row["created_at"].strftime("%Y-%m-%d") if row["created_at"] else "N/A",
                    "doc_type": row["doc_type"],
                    "content": content
                })

            search_results.sort(key=lambda x: x["relevance"], reverse=True)
            search_results = search_results[:10]
            context = "\n\n".join([res['content'] for res in search_results])

        except Exception as e:
            print(f"❌ MySQL 오류: {e}")
            visualizer.update_agent_status('rag_search', 'error')
            return {**state, "search_results": [], "context": ""}
        finally:
            if conn:
                conn.close()

        visualizer.update_agent_status('rag_search', 'completed', search_results)
        return {**state, "search_results": search_results, "context": context}

    except Exception as e:
        print(f"❌ RAG 검색 오류: {e}")
        visualizer.update_agent_status('rag_search', 'error')
        return {**state, "search_results": [], "context": ""}

def answer_generator_agent(state: AgentState):
    visualizer.update_agent_status('answer_generator', 'processing')
    
    if not state["search_results"]:
        visualizer.update_agent_status('answer_generator', 'completed')
        return {**state, "result": "관련 정보를 찾을 수 없습니다."}

    try:
        prompt = ChatPromptTemplate.from_template(
            """다음 문서 내용들을 바탕으로 사용자 질문에 답변해 주세요.
            - 문서에 직접 언급된 내용만을 바탕으로 답변해야 합니다.
            - 정보가 없는 경우 "정보가 없습니다"라고 명확히 답변해 주세요.
            - 항상 공손하고 전문적인 어조를 유지해 주세요.

            ### 검색 결과 요약:
            {search_summary}

            ### 문서 내용:
            {context}

            ### 사용자 질문:
            {query}

            ### 답변:
            찾은 문서중에 몇번째 정보가 가장 관련이 높은지를 상세히 답변해 주세요.
            (몇번째, 무슨파일인지, 어디에 있는지, 요약은 무엇인지 등)
            """
        )

        search_summary = "\n".join([
            f"{i+1}순위 ({res['relevance']}%): {res['file_name']} ({res['doc_type']}) - {res['summary']}"
            for i, res in enumerate(state["search_results"])
        ])

        chain = prompt | CHAT_LLM | StrOutputParser()
        result = chain.invoke({
            "search_summary": search_summary,
            "context": state["context"],
            "query": state["query"]
        })

        visualizer.update_agent_status('answer_generator', 'completed')
        return {**state, "result": result.strip()}

    except Exception as e:
        print(f"❌ 답변 생성 오류: {e}")
        visualizer.update_agent_status('answer_generator', 'error')
        return {**state, "result": "답변을 생성하는 중에 오류가 발생했습니다."}

def result_formatter_agent(state: AgentState):
    visualizer.update_agent_status('result_formatter', 'processing')
    
    if not state["search_results"]:
        visualizer.update_agent_status('result_formatter', 'completed')
        return state

    try:
        formatted_result = "\n🔍 검색 결과:\n"
        for i, res in enumerate(state["search_results"], 1):
            formatted_result += f"\n--- {i}순위 ({res['relevance']}%) ---\n"
            formatted_result += f"📄 파일명: {res['file_name']}\n"
            formatted_result += f"📁 위치: {res['file_location']}\n"
            formatted_result += f"📝 요약: {res['summary']}\n"
            formatted_result += f"🏷️ 유형: {res['doc_type']}\n"
            formatted_result += f"🕒 등록일: {res['created_at']}\n"

        formatted_result += f"\n💬 AI 답변:\n{state['result']}"

        visualizer.update_agent_status('result_formatter', 'completed')
        return {**state, "result": formatted_result}

    except Exception as e:
        print(f"❌ 결과 형식화 오류: {e}")
        visualizer.update_agent_status('result_formatter', 'error')
        return state

# ================================================================
# 7. LangGraph 구성
# ================================================================
graph = StateGraph(AgentState)
graph.add_node("extractor", extractor_agent)
graph.add_node("rag_search", rag_search_agent)
graph.add_node("answer_generator", answer_generator_agent)
graph.add_node("result_formatter", result_formatter_agent)

graph.set_entry_point("extractor")

graph.add_edge("extractor", "rag_search")
graph.add_edge("rag_search", "answer_generator")
graph.add_edge("answer_generator", "result_formatter")
graph.add_edge("result_formatter", END)

app = graph.compile()

# ================================================================
# 8. 실행 예시 (Jupyter Notebook)
# ================================================================
visualizer.show()

state = {
    "query": "한강에 버스가 있는 내용",
    "keywords": "",
    "search_results": [],
    "context": "",
    "result": ""
}

result = app.invoke(state)

print("\n" + "="*50)
print("📊 RAG 처리 완료! 최종 결과:")
print("="*50)
print(result["result"])

    'data': [{'marker': {'color': [gray, gray, gray, gray],
     …


📊 RAG 처리 완료! 최종 결과:

🔍 검색 결과:

--- 1순위 (100.0%) ---
📄 파일명: IT기사.docx
📁 위치: fileList/IT기사.docx
📝 요약: 홍민택 카카오 최고제품책임자(CPO)가 사내 공지를 통해 카카오톡 개편으로 인한 사용자 불만에 사과하며, 기존 서비스 유지와 함께 친구탭 피드 노출 방식 변경 및 인스타그램식 콘텐츠 분리 배치 계획을 밝혔다. 트래픽 감소는 없다고 전망하면서도 사용자 불편 최소화에 초점을 맞추겠다고 강조했다. 개편 배경은 카카오의 성장 정체 상황에서의 혁신 필요성과 기존 메...
🏷️ 유형: .docx
🕒 등록일: 2025-10-21

--- 2순위 (75.8%) ---
📄 파일명: 속도미달.docx
📁 위치: fileList/속도미달.docx
📝 요약: 서울시는 한강버스의 해상 시운전 결과를 알면서도 평균속력과 최대속력을 과장해 공개하며 시민들을 기만한 것으로 드러났다. 실제로 시운전에서 평균 최고속도는 15노트 미만으로, 계획된 17노트보다 낮게 나타났다. 서울시는 정식 운행 전 속도 조정에 대한 명확한 설명을 내놓지 않아 비판을 받았고, 이러한 문제점에도 불구하고 hurriedly 공식 발표를 진행하며...
🏷️ 유형: .docx
🕒 등록일: 2025-10-21

--- 3순위 (68.6%) ---
📄 파일명: IT과학정보.docx
📁 위치: fileList/IT과학정보.docx
📝 요약: SK텔레콤이 SK AX와 협력하여 개발한 AI 업무 지원 시스템 '에이닷 비즈'가 연말까지 SK그룹 내 25개 사에 도입될 예정이다. 이 시스템은 자연어 처리를 통해 회의록 작성 시간을 60%, 보고서 작성 시간을 40% 단축시키며, 정보 검색, 일정 관리, 채용 등 다양한 업무를 지원한다. 에이전트 빌더와 스토어 기능을 통해 IT 지식이 없는 구성원도 쉽...
🏷️ 유형: .docx
🕒 등록일: 2025-10-21

--- 4순위 (39.1%) ---
📄 파일명: 파렴치한.docx
📁 위치: fileList

In [35]:
# ================================================================
# 노트북용 RAG + 3D 시각화 통합 코드
# ================================================================
# 사용자의 질의로부터 관련 문서를 검색하고,
# 각 문서의 유사도(관련성)를 기준으로 3D 그래프 상에
# "질문 중심으로 가까운 순서"로 배치하는 통합 시각화 코드.
# ================================================================

from typing import TypedDict, List, Dict, Any
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langgraph.graph import StateGraph, END
import pymysql

import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import datetime

# ================================================================
# 1. 상태 정의
# ================================================================
class AgentState(TypedDict):
    query: str
    keywords: str
    search_results: List[Dict[str, Any]]
    context: str
    result: str

# ================================================================
# 2. DB 설정
# ================================================================
DB_CONFIG = {
    "host": "localhost",
    "user": "admin",
    "password": "1qazZAQ!",
    "db": "final",
    "charset": "utf8mb4",
}

# ================================================================
# 3. Chroma/LLM 설정
# ================================================================
CHROMA_PATH = "./rag_chroma/documents/title_summary/"
EMBEDDINGS = OllamaEmbeddings(model="exaone3.5:2.4b")
LLM = Ollama(model="exaone3.5:2.4b")
CHAT_LLM = ChatOllama(model="exaone3.5:2.4b", temperature=0.1)

# ================================================================
# 4. 3D 시각화 클래스
# ================================================================
class RAGNotebookVisualizer:
    def __init__(self):
        self.agent_positions = {
            "extractor": (0.0, 0.0, 0.0),
            "rag_search": (2.0, 0.0, 1.0),
            "answer_generator": (4.0, 0.0, 2.0),
            "result_formatter": (6.0, 0.0, 1.0),
        }
        self.agent_names = list(self.agent_positions.keys())
        self.agent_status = {n: "waiting" for n in self.agent_names}
        self.query_position = None

        self.fig3d = go.FigureWidget()
        self._init_scene()

        self.bar_fig = go.FigureWidget(
            data=[go.Bar(x=[], y=[], text=[], textposition="auto")],
            layout=go.Layout(title="검색 결과 관련성 (%)", height=260, margin=dict(t=40)),
        )

        self.progress = widgets.FloatProgress(
            value=0.0, min=0.0, max=100.0, description="진행률:", bar_style="info"
        )
        self.log = widgets.Textarea(
            value="",
            placeholder="로그가 여기에 표시됩니다.",
            layout=widgets.Layout(width="100%", height="600px"),
        )

        left = widgets.VBox([self.fig3d], layout=widgets.Layout(width="60%"))
        right = widgets.VBox([self.bar_fig, self.progress, self.log], layout=widgets.Layout(width="40%"))
        self.container = widgets.HBox([left, right])

    # --- 초기화 --------------------------------------------------
    def _init_scene(self):
        xs = [p[0] for p in self.agent_positions.values()]
        ys = [p[1] for p in self.agent_positions.values()]
        zs = [p[2] for p in self.agent_positions.values()]

        node_trace = go.Scatter3d(
            x=xs,
            y=ys,
            z=zs,
            mode="markers+text",
            marker=dict(size=14, color=["gray"] * len(self.agent_names), opacity=0.9),
            text=self.agent_names,
            textposition="top center",
            name="에이전트",
        )
        self.fig3d.add_trace(node_trace)

        for i in range(len(self.agent_names) - 1):
            a = self.agent_positions[self.agent_names[i]]
            b = self.agent_positions[self.agent_names[i + 1]]
            self.fig3d.add_trace(
                go.Scatter3d(
                    x=[a[0], b[0]], y=[a[1], b[1]], z=[a[2], b[2]],
                    mode="lines", line=dict(color="lightgray", width=4),
                    showlegend=False
                )
            )

        # --- 쿼리 노드
        self.query_trace_idx = len(self.fig3d.data)
        self.fig3d.add_trace(
            go.Scatter3d(
                x=[], y=[], z=[], mode="markers+text",
                marker=dict(size=20, color="orange", symbol="diamond"),
                text=[], textposition="top center", name="사용자 질문",
            )
        )

        # --- 쿼리-검색결과 연결선(빨강 점선)
        self.query_edge_trace_idx = len(self.fig3d.data)
        self.fig3d.add_trace(
            go.Scatter3d(
                x=[], y=[], z=[],
                mode="lines",
                line=dict(color="rgba(255,0,0,0.6)", width=2, dash="dot"),
                showlegend=False
            )
        )

        # --- 검색 결과 노드
        self.search_trace_idx = len(self.fig3d.data)
        self.fig3d.add_trace(
            go.Scatter3d(
                x=[], y=[], z=[], mode="markers+text",
                marker=dict(size=[], color=[], colorscale="Viridis",
                            cmin=0, cmax=100, opacity=0.8, showscale=True,
                            colorbar=dict(title="관련성 (%)")),
                text=[], textposition="top center", name="검색 결과"
            )
        )

        # --- 레이아웃
        self.fig3d.update_layout(
            title="🤖 RAG 처리 3D 시각화",
            scene=dict(
                xaxis=dict(title="", showticklabels=False),
                yaxis=dict(title="", showticklabels=False),
                zaxis=dict(title="⭐ 중요도 (관련성%)"),
                camera=dict(eye=dict(x=1.5, y=1.5, z=1.5)),
            ),
            height=600,
            margin=dict(l=0, r=0, t=40, b=0),
        )

    # --- UI 표시 -------------------------------------------------
    def show(self): display(self.container)

    # --- 로그 ----------------------------------------------------
    def _log(self, txt: str):
        stamp = datetime.datetime.now().strftime("%H:%M:%S")
        self.log.value += f"[{stamp}] {txt}\n"

    # --- 상태 업데이트 --------------------------------------------
    def update_agent_status(self, agent_name: str, status: str,
                            data: List[Dict[str, Any]] = None, query: str = None):
        if query:
            self._update_query_node(query)
        self._log(f"{agent_name} -> {status}")
        self.agent_status[agent_name] = status
        colors = [("yellow" if s=="processing" else
                   "green" if s=="completed" else
                   "red" if s=="error" else "gray")
                  for s in self.agent_status.values()]
        self.fig3d.data[0].marker.color = colors
        done = sum(1 for s in self.agent_status.values() if s=="completed")
        self.progress.value = done/len(self.agent_status)*100
        if data and agent_name=="rag_search":
            self.update_search_results(data)

    # --- 쿼리 노드 위치 ------------------------------------------
    def _update_query_node(self, query: str):
        x, y, z = -2.0, 0.0, 0.0   # 고정 위치
        self.query_position = (x, y, z)
        qtrace = self.fig3d.data[self.query_trace_idx]
        qtrace.x, qtrace.y, qtrace.z = [x], [y], [z]
        qtrace.text = [f"🔍 {query[:30]}"]

    # --- 검색 결과 업데이트 ---------------------------------------
    def update_search_results(self, results: List[Dict[str, Any]]):
        res = sorted(results, key=lambda r:r.get("relevance",0), reverse=True)
        if not self.query_position: base_x, base_y, base_z = (0,0,0)
        else: base_x, base_y, base_z = self.query_position

        xs, ys, zs, sizes, colors, texts = [],[],[],[],[],[]
        edge_xs, edge_ys, edge_zs = [],[],[]
        n = len(res)
        min_r, max_r = (0,100)
        for i, r in enumerate(res):
            rel = r.get("relevance",0)
            # 🔴 반경 반비례 (유사도 높을수록 가까움)
            radius = 0.5 + 2*(1 - rel/100)
            angle = (i / n) * 2*np.pi
            x = base_x + radius*np.cos(angle)
            y = base_y + radius*np.sin(angle)
            z = base_z + (rel/100)*2
            xs.append(x); ys.append(y); zs.append(z)
            colors.append(rel); sizes.append(max(6, rel/4))
            texts.append(f"{r.get('file_name','파일')}<br>{rel}%")
            edge_xs += [base_x, x, None]
            edge_ys += [base_y, y, None]
            edge_zs += [base_z, z, None]

        # --- 그래프 업데이트
        self.fig3d.data[self.query_edge_trace_idx].x = edge_xs
        self.fig3d.data[self.query_edge_trace_idx].y = edge_ys
        self.fig3d.data[self.query_edge_trace_idx].z = edge_zs
        s_trace = self.fig3d.data[self.search_trace_idx]
        s_trace.x, s_trace.y, s_trace.z = xs, ys, zs
        s_trace.marker.size, s_trace.marker.color, s_trace.text = sizes, colors, texts

        # --- 바 차트 업데이트
        fn = [r.get("file_name","") for r in res]
        rv = [r.get("relevance",0) for r in res]
        with self.bar_fig.batch_update():
            b=self.bar_fig.data[0]; b.x,b.y,b.text=fn,rv,[f"{v}%" for v in rv]
            b.marker.color,b.marker.colorscale,b.marker.cmin,b.marker.cmax,b.marker.showscale = rv,"Viridis",0,100,True

# ================================================================
# 5. 시각화 객체 생성
# ================================================================
visualizer = RAGNotebookVisualizer()

# ================================================================
# 6. 에이전트 함수
# ================================================================
def extractor_agent(state: AgentState):
    visualizer.update_agent_status("extractor","processing",query=state["query"])
    prompt = PromptTemplate.from_template(
        "사용자 질문에서 검색용 핵심 키워드를 쉼표로 구분해 출력:\n질문: {query}"
    )
    kw = LLM.invoke(prompt.format(query=state["query"]))
    visualizer.update_agent_status("extractor","completed")
    return {**state,"keywords":kw.strip()}

def rag_search_agent(state: AgentState):
    visualizer.update_agent_status("rag_search","processing")
    try:
        store=Chroma(persist_directory=CHROMA_PATH,embedding_function=EMBEDDINGS)
        docs=store.similarity_search_with_score(state["keywords"],k=10)
        if not docs:
            visualizer.update_agent_status("rag_search","completed",[])
            return {**state,"search_results":[],"context":""}
        best={}
        for d,s in docs:
            did=d.metadata.get("id")
            if not did: continue
            if did not in best or s<best[did][0]: best[did]=(s,d.page_content)
        scores=[s for s,_ in best.values()]; min_s,max_s=min(scores),max(scores)
        conn=None; results=[]
        conn=pymysql.connect(**DB_CONFIG)
        cur=conn.cursor(pymysql.cursors.DictCursor)
        for did,(s,content) in best.items():
            cur.execute("SELECT * FROM documents WHERE id=%s",(did,))
            row=cur.fetchone()
            if not row: continue
            rel=(1-(s-min_s)/(max_s-min_s))*100 if max_s!=min_s else 100
            results.append({
                "relevance":round(rel,1),
                "file_name":row["file_name"],
                "file_location":row["file_location"],
                "summary":row["summary"][:200]+"..." if row["summary"] else "요약 없음",
                "title":row["title"],
                "created_at":row["created_at"].strftime("%Y-%m-%d") if row["created_at"] else "N/A",
                "doc_type":row["doc_type"],
                "content":content})
        results.sort(key=lambda r:r["relevance"],reverse=True)
        context="\n\n".join(r["content"] for r in results[:10])
        visualizer.update_agent_status("rag_search","completed",results)
        return {**state,"search_results":results,"context":context}
    except Exception as e:
        print("❌ 오류:",e)
        visualizer.update_agent_status("rag_search","error")
        return {**state,"search_results":[],"context":""}
    finally:
        if "conn" in locals() and conn: conn.close()

def answer_generator_agent(state: AgentState):
    visualizer.update_agent_status("answer_generator","processing")
    if not state["search_results"]:
        visualizer.update_agent_status("answer_generator","completed")
        return {**state,"result":"관련 정보 없음"}
    summ="\n".join([f"{i+1} ({r['relevance']}%): {r['file_name']}" for i,r in enumerate(state["search_results"])])
    chain=ChatPromptTemplate.from_template(
        "검색 결과를 바탕으로 질문에 답하시오.\n검색 요약:\n{summ}\n문서내용:\n{context}\n질문:{query}\n답변:"
    )|CHAT_LLM|StrOutputParser()
    ans=chain.invoke({"summ":summ,"context":state["context"],"query":state["query"]})
    visualizer.update_agent_status("answer_generator","completed")
    return {**state,"result":ans.strip()}

def result_formatter_agent(state: AgentState):
    visualizer.update_agent_status("result_formatter","processing")
    text="\n🔍 검색 결과:\n"
    for i,r in enumerate(state["search_results"],1):
        text+=f"\n--- {i} ({r['relevance']}%) ---\n📄 {r['file_name']}\n📝 {r['summary']}\n"
    text+=f"\n💬 답변:\n{state['result']}"
    visualizer.update_agent_status("result_formatter","completed")
    return {**state,"result":text}

# ================================================================
# 7. LangGraph 설정 및 실행 예시
# ================================================================
graph=StateGraph(AgentState)
graph.add_node("extractor",extractor_agent)
graph.add_node("rag_search",rag_search_agent)
graph.add_node("answer_generator",answer_generator_agent)
graph.add_node("result_formatter",result_formatter_agent)
graph.set_entry_point("extractor")
graph.add_edge("extractor","rag_search")
graph.add_edge("rag_search","answer_generator")
graph.add_edge("answer_generator","result_formatter")
graph.add_edge("result_formatter",END)
app=graph.compile()

visualizer.show()
state={"query":"한강에 버스가 있는 내용","keywords":"","search_results":[],"context":"","result":""}
result=app.invoke(state)

print("="*50+"\n📊 RAG 완료 결과:\n"+result["result"])

    'data': [{'marker': {'color': ['gray', 'gray', 'gray', 'gray'…

📊 RAG 완료 결과:

🔍 검색 결과:

--- 1 (100.0%) ---
📄 속도미달.docx
📝 서울시는 한강버스의 해상 시운전 결과를 알면서도 평균속력과 최대속력을 과장해 공개하며 시민들을 기만한 것으로 드러났다. 실제로 시운전에서 평균 최고속도는 15노트 미만으로, 계획된 17노트보다 낮게 나타났다. 서울시는 정식 운행 전 속도 조정에 대한 명확한 설명을 내놓지 않아 비판을 받았고, 이러한 문제점에도 불구하고 hurriedly 공식 발표를 진행하며...

--- 2 (62.7%) ---
📄 IT기사.docx
📝 홍민택 카카오 최고제품책임자(CPO)가 사내 공지를 통해 카카오톡 개편으로 인한 사용자 불만에 사과하며, 기존 서비스 유지와 함께 친구탭 피드 노출 방식 변경 및 인스타그램식 콘텐츠 분리 배치 계획을 밝혔다. 트래픽 감소는 없다고 전망하면서도 사용자 불편 최소화에 초점을 맞추겠다고 강조했다. 개편 배경은 카카오의 성장 정체 상황에서의 혁신 필요성과 기존 메...

--- 3 (54.4%) ---
📄 세계기사.docx
📝 중국 상하이 출신의 금융학 석사 출신 청년 자오덴은 과도한 가족 압박과 외로움으로 인해 뉴질랜드 이주 후 다양한 도시에서 생활하며 학문적 성공을 이루었으나, 현재는 한 달에 약 2만원으로 생활하는 노숙자로 변모했다. 그는 깊이 느끼는 외로움과 부모와의 갈등 속에서 극단적인 저비용 생활을 선택해 자유를 추구하며, 의미 있는 활동으로 삶의 가치를 찾고 있다. ...

--- 4 (41.0%) ---
📄 파렴치한.docx
📝 지난달 20일, 유튜브 채널 '한문철 TV'에 제보된 사연에서, 주차 자리를 맡고 있던 아줌마와의 충돌로 인해 차주 A씨는 특수폭행으로 고소당했다. A씨는 아주머니가 자신의 차와 약간 접촉한 후에도 이를 무시하고 주차를 완료했으나, 5분 뒤 아주머니의 남편으로부터 고의적 폭행 주장에 대한 전화를 받고 고소를 당했다. A씨는 주차 시 주변 상황 확인 부족을 ...

--- 5 (32.3%) ---
📄 소름돋는정책원

# 질문 가운데 유사도에 따라 가까운 쪽으로 3D

In [38]:
# ================================================================
# 노트북용 RAG + 3D 시각화 통합 코드
# - 질문을 중앙에 고정하고, 관련성이 높을수록 질문에 가까이 배치하는 3D 시각화
# ================================================================

from typing import TypedDict, List, Dict, Any
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langgraph.graph import StateGraph, END
import pymysql

import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import datetime

# ================================================================
# 1. 상태 정의
# ================================================================
class AgentState(TypedDict):
    query: str
    keywords: str
    search_results: List[Dict[str, Any]]
    context: str
    result: str

# ================================================================
# 2. DB 설정
# ================================================================
DB_CONFIG = {
    "host": "localhost",
    "user": "admin",
    "password": "1qazZAQ!",
    "db": "final",
    "charset": "utf8mb4",
}

# ================================================================
# 3. Chroma/LLM 설정
# ================================================================
CHROMA_PATH = "./rag_chroma/documents/title_summary/"
EMBEDDINGS = OllamaEmbeddings(model="exaone3.5:2.4b")
LLM = Ollama(model="exaone3.5:2.4b")
CHAT_LLM = ChatOllama(model="exaone3.5:2.4b", temperature=0.1)

# ================================================================
# 4. 3D 시각화 클래스
# ================================================================
class RAGNotebookVisualizer:
    def __init__(self):
        # (기존 에이전트 좌표는 유지하되, query는 중앙(0,0,0)으로 표시)
        self.agent_positions = {
            "extractor": (0.0, -3.0, 0.0),
            "rag_search": (2.0, -3.0, 0.0),
            "answer_generator": (4.0, -3.0, 0.0),
            "result_formatter": (6.0, -3.0, 0.0),
        }
        self.agent_names = list(self.agent_positions.keys())
        self.agent_status = {n: "waiting" for n in self.agent_names}
        self.query_position = (0.0, 0.0, 0.0)  # 중앙 고정

        self.fig3d = go.FigureWidget()
        self._init_scene()

        self.bar_fig = go.FigureWidget(
            data=[go.Bar(x=[], y=[], text=[], textposition="auto")],
            layout=go.Layout(title="검색 결과 관련성 (%)", height=260, margin=dict(t=40)),
        )

        self.progress = widgets.FloatProgress(
            value=0.0, min=0.0, max=100.0, description="진행률:", bar_style="info"
        )
        self.log = widgets.Textarea(
            value="",
            placeholder="로그가 여기에 표시됩니다.",
            layout=widgets.Layout(width="100%", height="600px"),
        )

        left = widgets.VBox([self.fig3d], layout=widgets.Layout(width="60%"))
        right = widgets.VBox([self.bar_fig, self.progress, self.log], layout=widgets.Layout(width="40%"))
        self.container = widgets.HBox([left, right])

    def _init_scene(self):
        # 에이전트 노드 표시
        xs = [p[0] for p in self.agent_positions.values()]
        ys = [p[1] for p in self.agent_positions.values()]
        zs = [p[2] for p in self.agent_positions.values()]

        node_trace = go.Scatter3d(
            x=xs,
            y=ys,
            z=zs,
            mode="markers+text",
            marker=dict(size=12, color=["gray"] * len(self.agent_names), opacity=0.9),
            text=self.agent_names,
            textposition="top center",
            name="에이전트",
        )
        self.fig3d.add_trace(node_trace)

        # 에이전트 간 연결선
        for i in range(len(self.agent_names) - 1):
            a = self.agent_positions[self.agent_names[i]]
            b = self.agent_positions[self.agent_names[i + 1]]
            self.fig3d.add_trace(
                go.Scatter3d(
                    x=[a[0], b[0]],
                    y=[a[1], b[1]],
                    z=[a[2], b[2]],
                    mode="lines",
                    line=dict(color="lightgray", width=3),
                    showlegend=False,
                )
            )

        # 쿼리 노드 (중앙)
        self.query_trace_idx = len(self.fig3d.data)
        self.fig3d.add_trace(
            go.Scatter3d(
                x=[self.query_position[0]],
                y=[self.query_position[1]],
                z=[self.query_position[2]],
                mode="markers+text",
                marker=dict(size=24, color="orange", symbol="diamond", line=dict(width=2, color="darkorange")),
                text=[f"🔍 Query"],
                textposition="bottom center",
                name="사용자 질문",
                hovertemplate="<b>쿼리</b>: %{text}<extra></extra>",
            )
        )

        # 쿼리-검색결과 연결선 (빨강 점선)
        self.query_edge_trace_idx = len(self.fig3d.data)
        self.fig3d.add_trace(
            go.Scatter3d(
                x=[], y=[], z=[],
                mode="lines",
                line=dict(color="rgba(255,0,0,0.6)", width=2, dash="dot"),
                showlegend=False,
                hoverinfo="skip",
            )
        )

        # 검색 결과 노드 (업데이트시 채움)
        self.search_trace_idx = len(self.fig3d.data)
        self.fig3d.add_trace(
            go.Scatter3d(
                x=[], y=[], z=[],
                mode="markers+text",
                marker=dict(
                    size=[], color=[], colorscale="Viridis",
                    cmin=0, cmax=100, opacity=0.9, showscale=True,
                    colorbar=dict(title="관련성 (%)")
                ),
                text=[], textposition="top center", name="검색 결과"
            )
        )

        # 레이아웃: x/y 축 라벨 숨김, z축만 보이게
        self.fig3d.update_layout(
            title="🤖 RAG 처리 3D 시각화 (질문 중심, 가까울수록 유사도 높음)",
            scene=dict(
                xaxis=dict(title="", showticklabels=False, showgrid=False, zeroline=False),
                yaxis=dict(title="", showticklabels=False, showgrid=False, zeroline=False),
                zaxis=dict(title="거리(작을수록 유사도↑)"),
                camera=dict(eye=dict(x=1.5, y=1.5, z=1.2)),
            ),
            height=700,
            margin=dict(l=0, r=0, t=60, b=0),
        )

    def show(self):
        display(self.container)

    def _log(self, txt: str):
        stamp = datetime.datetime.now().strftime("%H:%M:%S")
        self.log.value += f"[{stamp}] {txt}\n"

    def update_agent_status(self, agent_name: str, status: str,
                            data: List[Dict[str, Any]] = None, query: str = None):
        # query 전달 시 중앙 쿼리 텍스트 업데이트
        if query is not None:
            self._update_query_node(query)
        self._log(f"{agent_name} -> {status}")
        if agent_name in self.agent_status:
            self.agent_status[agent_name] = status

        colors = [("yellow" if s == "processing" else "green" if s == "completed" else "red" if s == "error" else "gray")
                  for s in self.agent_status.values()]
        self.fig3d.data[0].marker.color = colors

        completed_count = sum(1 for s in self.agent_status.values() if s == "completed")
        self.progress.value = (completed_count / len(self.agent_status)) * 100

        if data and agent_name == "rag_search":
            self.update_search_results(data)

    def _update_query_node(self, query: str):
        # query 텍스트를 중앙에 업데이트 (중앙 좌표는 self.query_position)
        base_x, base_y, base_z = self.query_position
        qtext = f"🔍 {query[:40]}" if query else "🔍 Query"
        qtrace = self.fig3d.data[self.query_trace_idx]
        qtrace.x = [base_x]
        qtrace.y = [base_y]
        qtrace.z = [base_z]
        qtrace.text = [qtext]

    def update_search_results(self, results: List[Dict[str, Any]]):
        # results: list of dicts with 'relevance' and 'file_name' etc.
        res_sorted = sorted(results, key=lambda r: r.get("relevance", 0), reverse=True)
        n = len(res_sorted)
        if n == 0:
            # 빈 결과면 기존 trace 비움
            self.fig3d.data[self.query_edge_trace_idx].x = []
            self.fig3d.data[self.query_edge_trace_idx].y = []
            self.fig3d.data[self.query_edge_trace_idx].z = []
            self.fig3d.data[self.search_trace_idx].x = []
            self.fig3d.data[self.search_trace_idx].y = []
            self.fig3d.data[self.search_trace_idx].z = []
            return

        base_x, base_y, base_z = self.query_position

        # 배치 파라미터: 중앙에 가까울수록 관련성 높음
        min_radius = 0.15   # 가장 관련성 높은 문서의 최소 거리
        max_radius = 3.0    # 가장 관련성 낮은 문서의 최대 거리

        xs, ys, zs = [], [], []
        sizes, colors, texts = [], [], []
        edge_xs, edge_ys, edge_zs = [], [], []

        # golden angle 기반 방향 분포 (구면 좌표 사용)
        golden_angle = np.pi * (3 - np.sqrt(5))  # ~2.39996

        for i, item in enumerate(res_sorted):
            rel = float(item.get("relevance", 0.0))
            # radius: 관련성 높을수록 작음
            radius = min_radius + (1.0 - rel / 100.0) * (max_radius - min_radius)

            # 방향: 일정한 분포를 위해 theta/phi 사용
            theta = (i * golden_angle) % (2 * np.pi)
            # phi 분포 (0..pi) 균등화를 위해 다음 식 사용
            phi = np.arccos(1 - 2 * (i + 0.5) / max(n, 1))

            # 구면 좌표 -> 직교 좌표
            x = base_x + radius * np.sin(phi) * np.cos(theta)
            y = base_y + radius * np.sin(phi) * np.sin(theta)
            z = base_z + radius * np.cos(phi)

            xs.append(x); ys.append(y); zs.append(z)
            sizes.append(max(6, rel / 4.0))
            colors.append(rel)
            texts.append(f"{item.get('file_name','파일')}<br>{rel}%")

            # 쿼리(중앙)에서 각 노드로 연결선
            edge_xs.extend([base_x, x, None])
            edge_ys.extend([base_y, y, None])
            edge_zs.extend([base_z, z, None])

        # 업데이트: 연결선 (빨강 점선)
        e_trace = self.fig3d.data[self.query_edge_trace_idx]
        e_trace.x = edge_xs
        e_trace.y = edge_ys
        e_trace.z = edge_zs

        # 업데이트: 검색 결과 노드
        s_trace = self.fig3d.data[self.search_trace_idx]
        s_trace.x = xs
        s_trace.y = ys
        s_trace.z = zs
        s_trace.marker.size = sizes
        s_trace.marker.color = colors
        s_trace.text = texts

        # 막대 그래프 업데이트 (관련성 내림차순)
        file_names = [r.get("file_name", "") for r in res_sorted]
        relevances = [r.get("relevance", 0) for r in res_sorted]
        with self.bar_fig.batch_update():
            b = self.bar_fig.data[0]
            b.x = file_names
            b.y = relevances
            b.text = [f"{v}%" for v in relevances]
            b.marker.color = relevances
            b.marker.colorscale = "Viridis"
            b.marker.cmin = 0
            b.marker.cmax = 100
            b.marker.showscale = True

# ================================================================
# 5. 시각화 객체 생성
# ================================================================
visualizer = RAGNotebookVisualizer()

# ================================================================
# 6. 에이전트 함수
# ================================================================
def extractor_agent(state: AgentState):
    visualizer.update_agent_status("extractor", "processing", query=state["query"])
    prompt = PromptTemplate.from_template(
        "사용자 질문에서 검색용 핵심 키워드를 쉼표로 구분해 출력:\n질문: {query}"
    )
    kw = LLM.invoke(prompt.format(query=state["query"]))
    visualizer.update_agent_status("extractor", "completed")
    return {**state, "keywords": kw.strip()}

def rag_search_agent(state: AgentState):
    visualizer.update_agent_status("rag_search", "processing")
    try:
        store = Chroma(persist_directory=CHROMA_PATH, embedding_function=EMBEDDINGS)
        docs = store.similarity_search_with_score(state["keywords"], k=10)
        if not docs:
            visualizer.update_agent_status("rag_search", "completed", [])
            return {**state, "search_results": [], "context": ""}

        best = {}
        for d, s in docs:
            did = d.metadata.get("id")
            if not did:
                continue
            content = d.page_content.strip()
            if not content:
                continue
            # score: 작은 값이 더 유사한 경우(거리 등)인 경우 min 기준
            if did not in best or s < best[did][0]:
                best[did] = (s, content)

        if not best:
            visualizer.update_agent_status("rag_search", "completed", [])
            return {**state, "search_results": [], "context": ""}

        scores = [score for score, _ in best.values()]
        min_s, max_s = min(scores), max(scores)

        conn = pymysql.connect(**DB_CONFIG)
        cur = conn.cursor(pymysql.cursors.DictCursor)
        results = []
        for did, (s, content) in best.items():
            cur.execute("SELECT * FROM documents WHERE id = %s", (did,))
            row = cur.fetchone()
            if not row:
                continue
            if max_s != min_s:
                rel = (1 - (s - min_s) / (max_s - min_s)) * 100
            else:
                rel = 100.0
            results.append({
                "relevance": round(rel, 1),
                "file_name": row.get("file_name"),
                "file_location": row.get("file_location"),
                "summary": (row.get("summary")[:200] + "...") if row.get("summary") else "요약 없음",
                "title": row.get("title"),
                "created_at": row.get("created_at").strftime("%Y-%m-%d") if row.get("created_at") else "N/A",
                "doc_type": row.get("doc_type"),
                "content": content
            })
        if conn:
            conn.close()

        results.sort(key=lambda r: r["relevance"], reverse=True)
        context = "\n\n".join([r["content"] for r in results[:10]])
        visualizer.update_agent_status("rag_search", "completed", results)
        return {**state, "search_results": results, "context": context}
    except Exception as e:
        print("❌ RAG 검색 오류:", e)
        visualizer.update_agent_status("rag_search", "error")
        return {**state, "search_results": [], "context": ""}

def answer_generator_agent(state: AgentState):
    visualizer.update_agent_status("answer_generator", "processing")
    if not state["search_results"]:
        visualizer.update_agent_status("answer_generator", "completed")
        return {**state, "result": "관련 정보 없음"}
    search_summary = "\n".join([
        f"{i+1}위 ({r['relevance']}%): {r['file_name']} - {r['summary']}"
        for i, r in enumerate(state["search_results"])
    ])
    prompt = ChatPromptTemplate.from_template(
        "다음 검색 요약과 문서 내용을 보고 질문에 답하시오.\n검색 요약:\n{search_summary}\n문서내용:\n{context}\n질문:{query}\n답변:"
    )
    chain = prompt | CHAT_LLM | StrOutputParser()
    ans = chain.invoke({"search_summary": search_summary, "context": state["context"], "query": state["query"]})
    visualizer.update_agent_status("answer_generator", "completed")
    return {**state, "result": ans.strip()}

def result_formatter_agent(state: AgentState):
    visualizer.update_agent_status("result_formatter", "processing")
    out = "\n🔍 검색 결과:\n"
    for i, r in enumerate(state["search_results"], 1):
        out += f"\n--- {i}위 ({r['relevance']}%) ---\n📄 {r['file_name']}\n📝 {r['summary']}\n"
    out += f"\n💬 AI 답변:\n{state['result']}"
    visualizer.update_agent_status("result_formatter", "completed")
    return {**state, "result": out}

# ================================================================
# 7. LangGraph 설정 및 실행 예시
# ================================================================
graph = StateGraph(AgentState)
graph.add_node("extractor", extractor_agent)
graph.add_node("rag_search", rag_search_agent)
graph.add_node("answer_generator", answer_generator_agent)
graph.add_node("result_formatter", result_formatter_agent)
graph.set_entry_point("extractor")
graph.add_edge("extractor", "rag_search")
graph.add_edge("rag_search", "answer_generator")
graph.add_edge("answer_generator", "result_formatter")
graph.add_edge("result_formatter", END)
app = graph.compile()

# 시각화 표시 및 실행 예시
visualizer.show()
state = {"query": "한강에 버스가 있는 내용", "keywords": "", "search_results": [], "context": "", "result": ""}
result = app.invoke(state)

print("\n" + "=" * 60)
print("📊 RAG 처리 완료! 최종 결과:\n")
print(result["result"])

    'data': [{'marker': {'color': ['gray', 'gray', 'gray', 'gray'…


📊 RAG 처리 완료! 최종 결과:


🔍 검색 결과:

--- 1위 (100.0%) ---
📄 IT기사.docx
📝 홍민택 카카오 최고제품책임자(CPO)가 사내 공지를 통해 카카오톡 개편으로 인한 사용자 불만에 사과하며, 기존 서비스 유지와 함께 친구탭 피드 노출 방식 변경 및 인스타그램식 콘텐츠 분리 배치 계획을 밝혔다. 트래픽 감소는 없다고 전망하면서도 사용자 불편 최소화에 초점을 맞추겠다고 강조했다. 개편 배경은 카카오의 성장 정체 상황에서의 혁신 필요성과 기존 메...

--- 2위 (84.2%) ---
📄 속도미달.docx
📝 서울시는 한강버스의 해상 시운전 결과를 알면서도 평균속력과 최대속력을 과장해 공개하며 시민들을 기만한 것으로 드러났다. 실제로 시운전에서 평균 최고속도는 15노트 미만으로, 계획된 17노트보다 낮게 나타났다. 서울시는 정식 운행 전 속도 조정에 대한 명확한 설명을 내놓지 않아 비판을 받았고, 이러한 문제점에도 불구하고 hurriedly 공식 발표를 진행하며...

--- 3위 (54.0%) ---
📄 세계기사.docx
📝 중국 상하이 출신의 금융학 석사 출신 청년 자오덴은 과도한 가족 압박과 외로움으로 인해 뉴질랜드 이주 후 다양한 도시에서 생활하며 학문적 성공을 이루었으나, 현재는 한 달에 약 2만원으로 생활하는 노숙자로 변모했다. 그는 깊이 느끼는 외로움과 부모와의 갈등 속에서 극단적인 저비용 생활을 선택해 자유를 추구하며, 의미 있는 활동으로 삶의 가치를 찾고 있다. ...

--- 4위 (49.3%) ---
📄 IT과학정보.docx
📝 SK텔레콤이 SK AX와 협력하여 개발한 AI 업무 지원 시스템 '에이닷 비즈'가 연말까지 SK그룹 내 25개 사에 도입될 예정이다. 이 시스템은 자연어 처리를 통해 회의록 작성 시간을 60%, 보고서 작성 시간을 40% 단축시키며, 정보 검색, 일정 관리, 채용 등 다양한 업무를 지원한다. 에이전트 빌더와 스토어 기능을 통해 IT 지식이 없는 구성원도 쉽...

--- 5위 (41.9

# 시각화 테스트

In [58]:
# ================================================================
# 노트북용 RAG + 3D 시각화 통합 코드
# - 질문(Query)을 중앙(0,0,0)에 고정
# - 유사도(관련성)가 높을수록 중앙에 가깝게 배치 (Z축이 거리/깊이 역할)
# - 에이전트 시각화, 프로세스 로그, 진행률 바 제거
# ================================================================

from typing import TypedDict, List, Dict, Any
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
import pymysql

import plotly.graph_objects as go
import numpy as np
import ipywidgets as widgets
from IPython.display import display

# ================================================================
# 1. 상태 정의
# ================================================================
class AgentState(TypedDict):
    query: str
    keywords: str
    search_results: List[Dict[str, Any]]
    context: str
    result: str

# ================================================================
# 2. DB 설정
# ================================================================
DB_CONFIG = {
    "host": "localhost",
    "user": "admin",
    "password": "1qazZAQ!",
    "db": "final",
    "charset": "utf8mb4",
}

# ================================================================
# 3. Chroma/LLM 설정
# ================================================================
CHROMA_PATH = "./rag_chroma/documents/title_summary/"
EMBEDDINGS = OllamaEmbeddings(model="exaone3.5:2.4b")
LLM = Ollama(model="exaone3.5:2.4b")
CHAT_LLM = ChatOllama(model="exaone3.5:2.4b", temperature=0.1)

# ================================================================
# 4. 3D 시각화 클래스
# ================================================================
class RAGNotebookVisualizer:
    def __init__(self):
        # 에이전트 위치는 시각적 구분을 위해 유지하되, 메인 시각화에서는 숨김
        self.agent_positions = {
            "extractor": (0.0, -3.0, 0.0),
            "rag_search": (2.0, -3.0, 0.0),
            "answer_generator": (4.0, -3.0, 0.0),
            "result_formatter": (6.0, -3.0, 0.0),
        }
        self.agent_names = list(self.agent_positions.keys())
        self.agent_status = {n: "waiting" for n in self.agent_names}
        
        # 질문(쿼리)은 항상 중앙 (0, 0, 0)에 고정
        self.query_position = (0.0, 0.0, 0.0)

        self.fig3d = go.FigureWidget()
        self._init_scene()

        # 관련성 막대 그래프
        self.bar_fig = go.FigureWidget(
            data=[go.Bar(x=[], y=[], text=[], textposition="auto")],
            layout=go.Layout(title="검색 결과 관련성 (%)", height=260, margin=dict(t=40)),
        )

        # 로그 창 제거 (전체 UI에서 제거됨)
        
        left_panel = widgets.VBox([self.fig3d], layout=widgets.Layout(width="60%"))
        right_panel = widgets.VBox([self.bar_fig], layout=widgets.Layout(width="40%"))
        self.container = widgets.HBox([left_panel, right_panel])

    def _init_scene(self):
        """3D 장면 초기화"""
        # 1. 에이전트 노드 그리기 (숨기기 위해 비어있거나 투명하게 처리 가능하나, 여기서는 그냥 둡니다)
        xs = [p[0] for p in self.agent_positions.values()]
        ys = [p[1] for p in self.agent_positions.values()]
        zs = [p[2] for p in self.agent_positions.values()]

        self.fig3d.add_trace(go.Scatter3d(
            x=xs, y=ys, z=zs,
            mode="markers+text",
            marker=dict(size=10, color=["white"] * len(self.agent_names), opacity=0.0), # 에이전트 숨김
            text=["" if n != "rag_search" else "RAG" for n in self.agent_names], # rag_search만 살짝 보이게 할 수 있음 (선택)
            textposition="bottom center", name="에이전트", showlegend=False
        ))

        # 2. 쿼리 노드 (중앙)
        self.query_trace_idx = len(self.fig3d.data)
        self.fig3d.add_trace(go.Scatter3d(
            x=[0], y=[0], z=[0],
            mode="markers+text",
            marker=dict(size=15, color="orange", symbol="diamond", line=dict(width=2, color="darkorange")),
            text=["❓ Query"], textposition="bottom center", name="사용자 질문"
        ))

        # 3. 쿼리-결과 연결선 (빨간 점선)
        self.query_edge_trace_idx = len(self.fig3d.data)
        self.fig3d.add_trace(go.Scatter3d(
            x=[], y=[], z=[],
            mode="lines",
            line=dict(color="rgba(255,0,0,0.5)", width=2, dash="dot"),
            showlegend=False, hoverinfo="skip"
        ))

        # 4. 검색 결과 노드 (업데이트시 채움)
        self.search_trace_idx = len(self.fig3d.data)
        self.fig3d.add_trace(go.Scatter3d(
            x=[], y=[], z=[],
            mode="markers+text",
            marker=dict(
                size=[], color=[], colorscale="Viridis",
                cmin=0, cmax=100, opacity=0.9, showscale=True,
                colorbar=dict(title="관련성(%)")
            ),
            text=[], textposition="top center", name="검색 결과"
        ))

        # 5. 레이아웃 설정 (축 숨김)
        self.fig3d.update_layout(
            title="🔍 질문 기반 문서 배치 시각화 (가까울수록 유사도 ↑)",
            scene=dict(
                xaxis=dict(visible=False),
                yaxis=dict(visible=False),
                zaxis=dict(title="거리 (유사도)"),
                camera=dict(eye=dict(x=1.1, y=1.1, z=0.8)),
            ),
            height=600,
            margin=dict(l=0, r=0, t=40, b=0),
        )

    def show(self):
        display(self.container)

    def update_search_results(self, results: List[Dict[str, Any]]):
        """검색 결과를 3D 그래프와 막대 차트에 업데이트"""
        res_sorted = sorted(results, key=lambda x: x.get("relevance", 0), reverse=True)
        n = len(res_sorted)
        if n == 0: return

        bx, by, bz = self.query_position
        min_dist, max_dist = 0.2, 3.0 # 관련성 100%일 때의 최소 거리, 0%일 때의 최대 거리

        xs, ys, zs = [], [], []
        edge_xs, edge_ys, edge_zs = [], [], []
        sizes, colors, texts = [], [], []

        # 구면 좌표계를 이용해 배치
        golden_angle = np.pi * (3 - np.sqrt(5))
        for i, r in enumerate(res_sorted):
            rel = r.get("relevance", 0)
            # 유사도가 높을수록 (rel이 클수록) dist가 작아져 중앙에 가까워짐
            dist = min_dist + (1 - rel / 100) * (max_dist - min_dist)

            theta = i * golden_angle
            phi = np.arccos(1 - 2 * (i + 0.5) / n)

            x = bx + dist * np.sin(phi) * np.cos(theta)
            y = by + dist * np.sin(phi) * np.sin(theta)
            z = bz + dist * np.cos(phi)

            xs.append(x); ys.append(y); zs.append(z)
            edge_xs.extend([bx, x, None]); edge_ys.extend([by, y, None]); edge_zs.extend([bz, z, None])
            sizes.append(max(5, rel / 5))
            colors.append(rel)
            texts.append(f"{r.get('file_name','doc')}<br>{rel}%")

        # 3D 그래프 업데이트
        with self.fig3d.batch_update():
            self.fig3d.data[self.query_edge_trace_idx].x = edge_xs
            self.fig3d.data[self.query_edge_trace_idx].y = edge_ys
            self.fig3d.data[self.query_edge_trace_idx].z = edge_zs
            self.fig3d.data[self.search_trace_idx].x = xs
            self.fig3d.data[self.search_trace_idx].y = ys
            self.fig3d.data[self.search_trace_idx].z = zs
            self.fig3d.data[self.search_trace_idx].marker.size = sizes
            self.fig3d.data[self.search_trace_idx].marker.color = colors
            self.fig3d.data[self.search_trace_idx].text = texts

        # 막대 차트 업데이트
        with self.bar_fig.batch_update():
            self.bar_fig.data[0].x = [r.get("file_name", "") for r in res_sorted]
            self.bar_fig.data[0].y = [r.get("relevance", 0) for r in res_sorted]
            self.bar_fig.data[0].marker.color = [r.get("relevance", 0) for r in res_sorted]
            self.bar_fig.data[0].marker.colorscale = "Viridis"
            self.bar_fig.data[0].marker.showscale = True

# ================================================================
# 5. 시각화 객체 생성
# ================================================================
visualizer = RAGNotebookVisualizer()

# ================================================================
# 6. RAG 에이전트 함수 (에이전트 시각화 제거)
# ================================================================
def extractor_agent(state: AgentState):
    # 쿼리 업데이트 시점에만 시각화 업데이트를 위해 쿼리 전달
    visualizer.update_search_results(query=state["query"]) 
    
    prompt = PromptTemplate.from_template("질문에서 검색 키워드 추출 (쉼표구분):\n{query}")
    keywords = LLM.invoke(prompt.format(query=state["query"]))
    return {**state, "keywords": keywords.strip()}

def rag_search_agent(state: AgentState):
    try:
        vectorstore = Chroma(persist_directory=CHROMA_PATH, embedding_function=EMBEDDINGS)
        results = vectorstore.similarity_search_with_score(state["keywords"], k=10)

        if not results: return {**state, "search_results": [], "context": ""}

        best_doc_info = {}
        for doc, score in results:
            doc_id = doc.metadata.get("id")
            if doc_id:
                content = doc.page_content.strip()
                if content:
                    if doc_id not in best_doc_info or score < best_doc_info[doc_id][0]:
                        best_doc_info[doc_id] = (score, content)

        if not best_doc_info: return {**state, "search_results": [], "context": ""}

        scores = [score for score, _ in best_doc_info.values()]
        min_score, max_score = min(scores), max(scores)

        conn = pymysql.connect(**DB_CONFIG)
        cursor = conn.cursor(pymysql.cursors.DictCursor)
        search_results = []
        
        for doc_id, (score, content) in best_doc_info.items():
            cursor.execute("SELECT * FROM documents WHERE id = %s", (doc_id,))
            row = cursor.fetchone()
            if row:
                relevance = (1 - (score - min_score) / (max_score - min_score)) * 100 if max_score != min_score else 100
                search_results.append({
                    "relevance": round(relevance, 1),
                    "file_name": row["file_name"],
                    "file_location": row["file_location"],
                    "summary": row["summary"][:100] + "..." if row["summary"] else "",
                    "doc_type": row["doc_type"],
                    "content": content
                })
        
        conn.close()
        search_results.sort(key=lambda x: x["relevance"], reverse=True)
        context = "\n".join([r["content"] for r in search_results[:10]])
        
        visualizer.update_search_results(search_results)
        
        return {**state, "search_results": search_results, "context": context}
    
    except Exception as e:
        print(f"Error: {e}")
        return {**state, "search_results": [], "context": ""}

def answer_generator_agent(state: AgentState):
    if not state["search_results"]:
        return {**state, "result": "관련 정보 없음"}
    
    search_summary = "\n".join([
        f"- {r['file_name']} ({r['relevance']}%)" 
        for r in state["search_results"]
    ])
    
    prompt = ChatPromptTemplate.from_template(
        "문서를 참고하여 답변하시오.\n문서목록:\n{search_summary}\n\n질문: {query}"
    )
    chain = prompt | CHAT_LLM | StrOutputParser()
    result = chain.invoke({"search_summary": search_summary, "query": state["query"]})
    
    return {**state, "result": result.strip()}

def result_formatter_agent(state: AgentState):
    formatted_result = "🔍 검색 결과:\n"
    for i, r in enumerate(state["search_results"], 1):
        formatted_result += f"{i}. {r['file_name']} ({r['relevance']}%) - {r['summary']}\n"
    
    formatted_result += f"\n💬 답변:\n{state['result']}"
    return {**state, "result": formatted_result}

# ================================================================
# 7. LangGraph 구성 및 실행
# ================================================================
from langgraph.graph import StateGraph, END

graph = StateGraph(AgentState)
graph.add_node("extractor", extractor_agent)
graph.add_node("rag_search", rag_search_agent)
graph.add_node("answer_generator", answer_generator_agent)
graph.add_node("result_formatter", result_formatter_agent)

graph.set_entry_point("extractor")
graph.add_edge("extractor", "rag_search")
graph.add_edge("rag_search", "answer_generator")
graph.add_edge("answer_generator", "result_formatter")
graph.add_edge("result_formatter", END)

app = graph.compile()

# ================================================================
# 8. 실행 예시
# ================================================================
visualizer.show()
state = {"query": "한강에 버스가 있는 내용", "keywords": "", "search_results": [], "context": "", "result": ""}
result = app.invoke(state)

print("\n" + "="*50)
print("최종 결과:")
print("="*50)
print(result["result"])

    'data': [{'marker': {'color': ['white', 'white', 'white', 'wh…

TypeError: RAGNotebookVisualizer.update_search_results() got an unexpected keyword argument 'query'